# Project 2 - Classification and Regression -- 2013/2014 CitiBike-NYC Data
**Michael Smith, Alex Frye, Chris Boomhower ----- 3/08/2017**

<img src="https://github.com/msmith-ds/DataMining/blob/master/Project2_Full/Images/Citi-Bike.jpg?raw=true" width="400">

<center>Image courtesy of http://newyorkeronthetown.com/, 2017</center>

### Introduction
*** Describe the purpose of the model you are about to build ***

The data set again selected by our group for Lab 2 consists of [Citi Bike trip history](https://www.citibikenyc.com/system-data) data collected and released by NYC Bike Share, LLC and Jersey Bike Share, LLC under Citi Bike's [NYCBS Data Use Policy](https://www.citibikenyc.com/data-sharing-policy). Citi Bike is America's largest bike share program, with 10,000 bikes and 600 stations across Manhattan, Brooklyn, Queens, and Jersey City... 55 neighborhoods in all. As such, our data set's trip history includes all rental transactions conducted within the NYC Citi Bike system from July 1st, 2013 to February 28th, 2014. These transactions amount to 5,562,293 trips within this time frame. The original data set includes 15 attributes. In addition to these 15, our team was able to derive 15 more attributes for use in our classification efforts, some attributes of which are NYC weather data which come from [Carbon Dioxide Information Analysis Center (CDIAC)](http://cdiac.ornl.gov/cgi-bin/broker?_PROGRAM=prog.climsite_daily.sas&_SERVICE=default&id=305801&_DEBUG=0). These data are merged with the Citi Bike data to provide environmental insights into rider behavior.

The trip data was collected via Citi Bike's check-in/check-out system among 330 of its stations in the NYC system as part of its transaction history log. While the non-publicized data likely includes further particulars such as rider payment details, the publicized data is anonymized to protect rider identity while simultaneously offering bike share transportation insights to urban developers, engineers, academics, statisticians, and other interested parties. The CDIAC data, however, was collected by the Department of Energy's Oak Ridge National Laboratory for research into global climate change. While basic weather conditions are recorded by CDIAC, as included in our fully merged data set, the organization also measures atmospheric carbon dioxide and other radiatively active gas levels to conduct their research efforts.

Our team has taken particular interest in this data set as some of our team members enjoy both recreational and commute cycling. By combining basic weather data with Citi Bike's trip data, **our intent in this lab is to: 1) Predict whether riders are more likely to be (or become) Citi Bike subscribers based on ride environmental conditions, the day of the week for his/her trip, trip start and end locations, the general time of day (i.e. morning, midday, afternoon, evening, night) of his/her trip, his/her age and gender, etc., and 2) predict a rider's trip duration as a function of the aforementioned variables.** Due to the exhaustive number of observations in the original data set (5,562,293), a sample of 500,000 is selected to achieve these goals (as described further in the sections below). By leveraging Stratified 10-Fold Cross Validation, we expect to be able to derive dependable and accurate user type and ride duration prediction models for which accuracy and performance will be discussed in detail.

### Load the Data

##### Compiling Multiple Data Sources
To begin our analysis, we need to load the data from our source .csv files. Steps taken to pull data from the various source files are as follows:
- For each file from CitiBike, we process each line appending manually computed columns [LinearDistance, DayOfWeek, TimeOfDay, & HolidayFlag]. 
- Similarly, we load our weather data .csv file.
- With both source file variables gathered, we append the weather data to our CitiBike data by matching on the date.
- To avoid a 2 hour run-time in our analysis every execution, we load the final version of the data into .CSV files. Each file consists of 250,000 records to reduce file size for GitHub loads.
- All above logic is skipped if the file "Compiled Data/dataset1.csv" already exists.

Below you will see this process, as well as import/options for needed python modules throughout this analysis.

In [ ]:
import os
from geopy.distance import vincenty
import holidays
from datetime import datetime
from dateutil.parser import parse
import glob
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.ensemble  import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

%load_ext memory_profiler

plt.rcParams['figure.figsize'] = (12, 6)

pd.options.mode.chained_assignment = None

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
############################################################
# Load & Merge Data from Source Files
# Parse into Compiled Files
############################################################

starttime = datetime.now()
print('Starting Source Data Load & Merge Process. \n'
      'Start Time: ' + str(starttime))

if os.path.isfile("Compiled Data/dataset1.csv"):
    print("Found the File!")
else:
    citiBikeDataDirectory = "Citi Bike Data"
    citiBikeDataFileNames = [
        "2013-07 - Citi Bike trip data - 1.csv",
        "2013-07 - Citi Bike trip data - 2.csv",
        "2013-08 - Citi Bike trip data - 1.csv",
        "2013-08 - Citi Bike trip data - 2.csv",
        "2013-09 - Citi Bike trip data - 1.csv",
        "2013-09 - Citi Bike trip data - 2.csv",
        "2013-10 - Citi Bike trip data - 1.csv",
        "2013-10 - Citi Bike trip data - 2.csv",
        "2013-11 - Citi Bike trip data - 1.csv",
        "2013-11 - Citi Bike trip data - 2.csv",
        "2013-12 - Citi Bike trip data.csv",
        "2014-01 - Citi Bike trip data.csv",
        "2014-02 - Citi Bike trip data.csv"
    ]

    weatherDataFile = "Weather Data/NY305801_9255_edited.txt"

    citiBikeDataRaw = []

    for file in citiBikeDataFileNames:
        print(file)
        filepath = citiBikeDataDirectory + "/" + file
        with open(filepath) as f:
            lines = f.read().splitlines()
            lines.pop(0)  # get rid of the first line that contains the column names
            for line in lines:
                line = line.replace('"', '')
                line = line.split(",")
                sLatLong = (line[5], line[6])
                eLatLong = (line[9], line[10])

                distance = vincenty(sLatLong, eLatLong).miles
                line.extend([distance])

                ## Monday       = 0
                ## Tuesday      = 1
                ## Wednesday    = 2
                ## Thursday     = 3
                ## Friday       = 4
                ## Saturday     = 5
                ## Sunday       = 6
                if parse(line[1]).weekday() == 0:
                    DayOfWeek = "Monday"
                elif parse(line[1]).weekday() == 1:
                    DayOfWeek = "Tuesday"
                elif parse(line[1]).weekday() == 2:
                    DayOfWeek = "Wednesday"
                elif parse(line[1]).weekday() == 3:
                    DayOfWeek = "Thursday"
                elif parse(line[1]).weekday() == 4:
                    DayOfWeek = "Friday"
                elif parse(line[1]).weekday() == 5:
                    DayOfWeek = "Saturday"
                else:
                    DayOfWeek = "Sunday"
                line.extend([DayOfWeek])

                ##Morning       5AM-10AM
                ##Midday        10AM-2PM
                ##Afternoon     2PM-5PM
                ##Evening       5PM-10PM
                ##Night         10PM-5AM

                if parse(line[1]).hour >= 5 and parse(line[1]).hour < 10:
                    TimeOfDay = 'Morning'
                elif parse(line[1]).hour >= 10 and parse(line[1]).hour < 14:
                    TimeOfDay = 'Midday'
                elif parse(line[1]).hour >= 14 and parse(line[1]).hour < 17:
                    TimeOfDay = 'Afternoon'
                elif parse(line[1]).hour >= 17 and parse(line[1]).hour < 22:
                    TimeOfDay = 'Evening'
                else:
                    TimeOfDay = 'Night'
                line.extend([TimeOfDay])

                ## 1 = Yes
                ## 0 = No
                if parse(line[1]) in holidays.UnitedStates():
                    holidayFlag = "1"
                else:
                    holidayFlag = "0"
                line.extend([holidayFlag])

                citiBikeDataRaw.append(line)
            del lines

    with open(weatherDataFile) as f:
        weatherDataRaw = f.read().splitlines()
        weatherDataRaw.pop(0)  # again, get rid of the column names
        for c in range(len(weatherDataRaw)):
            weatherDataRaw[c] = weatherDataRaw[c].split(",")
            # Adjust days and months to have a leading zero so we can capture all the data
            if len(weatherDataRaw[c][2]) < 2:
                weatherDataRaw[c][2] = "0" + weatherDataRaw[c][2]
            if len(weatherDataRaw[c][0]) < 2:
                weatherDataRaw[c][0] = "0" + weatherDataRaw[c][0]

    citiBikeData = []

    while (citiBikeDataRaw):
        instance = citiBikeDataRaw.pop()
        date = instance[1].split(" ")[0].split("-")  # uses the start date of the loan
        for record in weatherDataRaw:
            if (str(date[0]) == str(record[4]) and str(date[1]) == str(record[2]) and str(date[2]) == str(record[0])):
                instance.extend([record[5], record[6], record[7], record[8], record[9]])
                citiBikeData.append(instance)

    del citiBikeDataRaw
    del weatherDataRaw

    # Final Columns:
    #  0 tripduration
    #  1 starttime
    #  2 stoptime
    #  3 start station id
    #  4 start station name
    #  5 start station latitude
    #  6 start station longitude
    #  7 end station id
    #  8 end station name
    #  9 end station latitude
    # 10 end station longitude
    # 11 bikeid
    # 12 usertype
    # 13 birth year
    # 14 gender
    # 15 start/end station distance
    # 16 DayOfWeek
    # 17 TimeOfDay
    # 18 HolidayFlag
    # 19 PRCP
    # 20 SNOW
    # 21 TAVE
    # 22 TMAX
    # 23 TMIN

    maxLineCount = 250000
    lineCounter = 1
    fileCounter = 1
    outputDirectoryFilename = "Compiled Data/dataset"
    f = open(outputDirectoryFilename + str(fileCounter) + ".csv", "w")
    for line in citiBikeData:
        if lineCounter == 250000:
            print(f)
            f.close()
            lineCounter = 1
            fileCounter = fileCounter + 1
            f = open(outputDirectoryFilename + str(fileCounter) + ".csv", "w")
        f.write(",".join(map(str, line)) + "\n")
        lineCounter = lineCounter + 1

    del citiBikeData

endtime = datetime.now()
print('Ending Source Data Load & Merge Process. \n'
      'End Time: ' + str(starttime) + '\n'
                                      'Total RunTime: ' + str(endtime - starttime))

##### Loading the Compiled Data from CSV

Now that we have compiled data files from both CitiBike and the weather data, we want to load that data into a Pandas dataframe for analysis. We iterate and load each file produced above, then assign each column with their appropriate data types. Additionally, we compute the Age Column after producing a default value for missing "Birth Year" values. This is discussed further in the Data Preparation 1 section.

In [ ]:
%%time
############################################################
# Load the Compiled Data from CSV
############################################################

# Create CSV Reader Function and assign column headers
def reader(f, columns):
    d = pd.read_csv(f)
    d.columns = columns
    return d


# Identify All CSV FileNames needing to be loaded
path = r'Compiled Data'
all_files = glob.glob(os.path.join(path, "*.csv"))

# Define File Columns
columns = ["tripduration", "starttime", "stoptime", "start_station_id", "start_station_name",
           "start_station_latitude",
           "start_station_longitude", "end_station_id", "end_station_name", "end_station_latitude",
           "end_station_longitude", "bikeid", "usertype", "birth year", "gender", "LinearDistance", "DayOfWeek",
           "TimeOfDay", "HolidayFlag", "PRCP", "SNOW", "TAVE", "TMAX", "TMIN"]

# Load Data
CitiBikeDataCompiled = pd.concat([reader(f, columns) for f in all_files])

# Replace '\N' Birth Years with Zero Values
CitiBikeDataCompiled["birth year"] = CitiBikeDataCompiled["birth year"].replace(r'\N', '0')

# Convert Columns to Numerical Values
CitiBikeDataCompiled[['tripduration', 'birth year', 'LinearDistance', 'PRCP', 'SNOW', 'TAVE', 'TMAX', 'TMIN']] \
    = CitiBikeDataCompiled[['tripduration', 'birth year', 'LinearDistance', 'PRCP', 'SNOW', 'TAVE', 'TMAX',
                            'TMIN']].apply(pd.to_numeric)

# Convert Columns to Date Values
CitiBikeDataCompiled[['starttime', 'stoptime']] \
    = CitiBikeDataCompiled[['starttime', 'stoptime']].apply(pd.to_datetime)

# Compute Age: 0 Birth Year = 0 Age ELSE Compute Start Time Year Minus Birth Year
CitiBikeDataCompiled["Age"] = np.where(CitiBikeDataCompiled["birth year"] == 0, 0,
                                       CitiBikeDataCompiled["starttime"].dt.year - CitiBikeDataCompiled[
                                           "birth year"])

# Convert Columns to Str Values
CitiBikeDataCompiled[['start_station_id', 'end_station_id', 'bikeid', 'HolidayFlag', 'gender']] \
    = CitiBikeDataCompiled[['start_station_id', 'end_station_id', 'bikeid', 'HolidayFlag', 'gender']].astype(str)

In [ ]:
%%time
print(len(CitiBikeDataCompiled))
display(CitiBikeDataCompiled.head())

## Data Preparation Part 1 - Define and prepare class variables

##### Measurable Data Quality Factors
When analyzing our final dataset for accurate measures, there are a few key factors we can easily verify/research:
- Computational Accuracy: Ensure data attributes added by computation are correct
    + TimeOfDay
    + DayOfWeek        
    + HolidayFlag
    
- Missing Data from Source
- Duplicate Data from Source
- Outlier Detection
- Sampling to 500,000 Records for further analysis

##### Immesurable Data Quality Factors
Although we are able to research these many factors, one computation may still be lacking information in this dataset. Our LinearDistance attribute computes the distance from  one lat/long coordinate to another. This attribute does not however tell us the 'true' distance a biker traveled before returning the bike. Some bikers may be biking for exercise around the city with various turns and loops, whereas others travel the quickest path to their destination. Because our dataset limits us to start and end locations, we do not have enough information to accurately compute distance traveled. Because of this, we have named the attribute "LinearDistance" rather than "DistanceTraveled".

Below we will walk through the process of researching the 'Measureable' data quality factors mentioned above.

###### Computational Accuracy:TimeOfDay
To help mitigate challenges with time series data, we have chosen to break TimeOfDay into 5 categories.
These Categories are broken down below:
- Morning       5  AM  -  10 AM
- Midday        10 AM  -  2  PM
- Afternoon     2  PM  -  5  PM
- Evening       5  PM  -  10 PM
- Night         10 PM  -  5  AM

To ensure that these breakdowns are accurately computed, we pulled the distinct list of TimeOfDay assignments by starttime hour. Looking at the results below, we can verify that this categorization is correctly being assigned.

In [ ]:
%%time
    # Compute StartHour from StartTime
CitiBikeDataCompiled["StartHour"] = CitiBikeDataCompiled["starttime"].dt.hour

    # Compute Distinct Combinations of StartHour and TimeOfDay
DistinctTimeOfDayByHour = CitiBikeDataCompiled[["StartHour", "TimeOfDay"]].drop_duplicates().sort_values("StartHour")

    # Print
display(DistinctTimeOfDayByHour)

    #Clean up Variables
del CitiBikeDataCompiled["StartHour"]

###### Computational Accuracy:DayOfWeek
In order to verify our computed DayOfWeek column, we have chosen one full week from 12/22/2013 - 12/28/2013 to validate. Below is a calendar image of this week to baseline our expected results:

<img src="https://github.com/msmith-ds/DataMining/blob/master/Project2_Full/Images/Dec_2013_Calendar.png?raw=true" width="300">

To verify these 7 days, we pulled the distinct list of DayOfWeek assignments by StartDate (No Time). If we can verify one full week, we may justify that the computation is correct across the entire dataset. Looking at the results below, we can verify that this categorization is correctly being assigned.

In [ ]:
%%time
    # Create DataFrame for StartTime, DayOfWeek within Date Threshold
CitiBikeDayOfWeekTest = CitiBikeDataCompiled[(CitiBikeDataCompiled['starttime'].dt.year == 2013)
                                             & (CitiBikeDataCompiled['starttime'].dt.month == 12)
                                             & (CitiBikeDataCompiled['starttime'].dt.day >= 22)
                                             & (CitiBikeDataCompiled['starttime'].dt.day <= 28)][
    ["starttime", "DayOfWeek"]]

    # Create FloorDate Variable as StartTime without the timestamp
CitiBikeDayOfWeekTest["StartFloorDate"] = CitiBikeDayOfWeekTest["starttime"].dt.strftime('%m/%d/%Y')

    # Compute Distinct combinations
DistinctDayOfWeek = CitiBikeDayOfWeekTest[["StartFloorDate", "DayOfWeek"]].drop_duplicates().sort_values(
    "StartFloorDate")

    #Print
display(DistinctDayOfWeek)

    # Clean up Variables
del CitiBikeDayOfWeekTest
del DistinctDayOfWeek

###### Computational Accuracy:HolidayFlag
Using the same week as was used to verify DayOfWeek, w can test whether HolidayFlag is set correctly for the Christmas Holiday. We pulled the distinct list of HolidayFlag assignments by StartDate (No Time). If we can verify one holiday, we may justify that the computation is correct across the entire dataset. Looking at the results below, we expect to see HolidayFlag = 1 only for 12/25/2013.

In [ ]:
%%time
    # Create DataFrame for StartTime, HolidayFlag within Date Threshold
CitiBikeHolidayFlagTest = CitiBikeDataCompiled[(CitiBikeDataCompiled['starttime'].dt.year == 2013)
                                             & (CitiBikeDataCompiled['starttime'].dt.month == 12)
                                             & (CitiBikeDataCompiled['starttime'].dt.day >= 22)
                                             & (CitiBikeDataCompiled['starttime'].dt.day <= 28)][
    ["starttime", "HolidayFlag"]]

    # Create FloorDate Variable as StartTime without the timestamp
CitiBikeHolidayFlagTest["StartFloorDate"] = CitiBikeHolidayFlagTest["starttime"].dt.strftime('%m/%d/%Y')

    # Compute Distinct combinations
DistinctHolidayFlag = CitiBikeHolidayFlagTest[["StartFloorDate", "HolidayFlag"]].drop_duplicates().sort_values(
    "StartFloorDate")
    
    #Print
display(DistinctHolidayFlag)
    
    # Clean up Variables
del CitiBikeHolidayFlagTest
del DistinctHolidayFlag


###### Missing Data from Source
Accounting for missing data is a crucial part of our analysis. At first glance, it is very apparent that we have a large amount of missing data in the Gender and Birth Year attributes from our source CitiBike Data. We have already had to handle for missing Birth Year attributes while computing "Age" in our Data Load from CSV section of this paper. This was done to create a DEFAULT value of (0), such that future computations do not result in NA values as well. Gender has also already accounted for missing values with a default value of (0) by the source data. Although we have handled these missing values with a default, we want to ensure that we 'need' these records for further analysis - or if we may remove them from the dataset. Below you will see a table showing the frequency of missing values(or forced default values) by usertype. We noticed that of the 4,881,384 Subscribing Members in our dataset, only 295 of them were missing Gender information, whereas out of the  680,909 Customer Users (Non-Subscribing), there was only one observation where we had complete information for both Gender and Birth Year. This quickly told us that removing records with missing values is NOT an option, since we would lose data for our entire Customer Usertype. These attributes, as well as Age (Computed from birth year) will serve as difficult for use in a classification model attempting to predict usertype. 

We have also looked at all other attributes, and verified that there are no additional missing values in our dataset. A missing value matrix was produced to identify if there were any gaps in our data across all attributes. Due to the conclusive results in our data, no missing values present, we removed this lackluster visualization from the report.

In [ ]:
%%time
NADatatestData = CitiBikeDataCompiled[["usertype","gender", "birth year"]]

NADatatestData["GenderISNA"] = np.where(CitiBikeDataCompiled["gender"] == '0', 1, 0)
NADatatestData["BirthYearISNA"] = np.where(CitiBikeDataCompiled["birth year"] == 0, 1,0)

NAAggs = pd.DataFrame({'count' : NADatatestData.groupby(["usertype","GenderISNA", "BirthYearISNA"]).size()}).reset_index()

display(NAAggs)

del NAAggs

###### Duplicate Data from Source
To ensure that there are no duplicate records in our datasets, we ensured that the number of records before and after removing potential duplicates were equal to each other. This test passed, thus we did not need any alterations to the dataset based on duplicate records.

In [ ]:
%%time
len(CitiBikeDataCompiled) == len(CitiBikeDataCompiled.drop_duplicates())

###### Outlier Detection

**Trip Duration**

In analyzing a Box Plot on trip duration values, we find extreme outliers present. With durations reaching up to 72 days in the most extreme instance, our team decided to rule out any observation with a duration greater than a 24 hour period. The likelihood of an individual sleeping overnight after their trip with the bike still checked out is much higher after the 24 hour period. This fact easily skews the results of this value, potentially hurting any analysis done. We move forward with removing a total of 457 observations based on trip duration greater than 24 hours (86,400 seconds).

In [ ]:
%%time
%matplotlib inline

#CitiBikeDataCompiledBackup = CitiBikeDataCompiled
#CitiBikeDataCompiled = CitiBikeDataCompiledBackup

    # BoxPlot tripDuration - Heavy Outliers!
sns.boxplot(y = "tripduration", data = CitiBikeDataCompiled)
sns.despine()
    
    # How Many Greater than 24 hours?
print(len(CitiBikeDataCompiled[CitiBikeDataCompiled["tripduration"]>86400]))

    # Remove > 24 Hours
CitiBikeDataCompiled = CitiBikeDataCompiled[CitiBikeDataCompiled["tripduration"]<86400]

Once outliers are removed, we run the boxplot again, still seeing skewness in results. To try to mitigate this left-skew distribution, we decide to take a log transform on this attribute. 

In [ ]:
%%time
%matplotlib inline

    # BoxPlot Trip Duration AFTER removal of outliers
sns.boxplot(y = "tripduration", data = CitiBikeDataCompiled)
sns.despine()

    # Log Transform Column Added
CitiBikeDataCompiled["tripdurationLog"] = CitiBikeDataCompiled["tripduration"].apply(np.log)


In [ ]:
%%time
%matplotlib inline

    # BoxPlot TripDurationLog
sns.boxplot(y = "tripdurationLog", data = CitiBikeDataCompiled)
sns.despine()

**Age**

Similarly, we look at the distribution of Age in our dataset. Interestingly, it seems we have several outlier observations logging their birth year far enough back to cause their age to compute as 115 years old. Possible reasons for these outlier ages could be data entry errors by those who do not enjoy disclosing personal information, or possibly account sharing between a parent and a child - rendering an inaccurate data point to those actually taking the trip. Our target demographic for this study are those individuals under 65 years of age, given that they are the likely age groups to be in better physical condition for the bike share service. Given this target demographic, and the poor entries causing extreme outliers, we have chosen to limit out dataset to observations up to 65 years of age. This change removed an additional 53824 records from the dataset.

In [ ]:
%%time
%matplotlib inline

    # BoxPlot Age - Outliers!
sns.boxplot(y = "Age", data = CitiBikeDataCompiled[CitiBikeDataCompiled["Age"]!= 0])
sns.despine()
    
    # How Many Greater than 65 years old?
print(len(CitiBikeDataCompiled[CitiBikeDataCompiled["Age"]>65]))

    # Remove > 65 years old
CitiBikeDataCompiled = CitiBikeDataCompiled[CitiBikeDataCompiled["Age"]<=65]


In [ ]:
%%time
%matplotlib inline

    # BoxPlot Age - removed Outliers!
sns.boxplot(y = "Age", data = CitiBikeDataCompiled[CitiBikeDataCompiled["Age"]!= 0])
sns.despine()

###### Record Sampling to 500,000 Records
Given the extremely large volume of data collected, we have have decided to try to sample down to ~ 1/10th of the original dataset for a total of 500,000 records. Before taking this action, however, we wanted to ensure that we keep data proportions reasonable for analysis and ensure we do not lose any important demographic in our data.

Below we compute the percentage of our Dataset that comprises of Customers vs. Subscribers. We note, that 87.6% of the data consists of Subscriber users whereas the remaining 12.4% resemble Customers. 

In [ ]:
%%time
%matplotlib inline
UserTypeDist = pd.DataFrame({'count' : CitiBikeDataCompiled.groupby(["usertype"]).size()}).reset_index()
display(UserTypeDist)

UserTypeDist.plot.pie(y = 'count', labels = ['Customer', 'Subscriber'], autopct='%1.1f%%')

In our Sample Dataset for this analysis, we have chosen to oversample the Customer observations to force a 50/50 split between the two classifications. This will help reduce bias in the model towards Subscribers simply due to the distribution of data in the sample.

We are able to compute the sample size for each usertype and then take a random sample within each group. Below you will see that our sampled distribution matches the chosen 50/50 split between Customers and Subscriber Usertypes. 

In [ ]:
%%time
SampleSize = 500000

CustomerSampleSize_Seed   = int(round(SampleSize * 50.0 / 100.0,0))
SubscriberSampleSize_Seed = int(round(SampleSize * 50.0 / 100.0,0))

CitiBikeCustomerDataSampled = CitiBikeDataCompiled[CitiBikeDataCompiled["usertype"] == 'Customer'].sample(n=CustomerSampleSize_Seed, replace = False, random_state = CustomerSampleSize_Seed)
CitiBikeSubscriberDataSampled = CitiBikeDataCompiled[CitiBikeDataCompiled["usertype"] == 'Subscriber'].sample(n=SubscriberSampleSize_Seed, replace = False, random_state = SubscriberSampleSize_Seed)

CitiBikeDataSampled_5050 = pd.concat([CitiBikeCustomerDataSampled,CitiBikeSubscriberDataSampled])

print(len(CitiBikeDataSampled_5050))

UserTypeDist = pd.DataFrame({'count' : CitiBikeDataSampled_5050.groupby(["usertype"]).size()}).reset_index()
display(UserTypeDist)

UserTypeDist.plot.pie(y = 'count', labels = ['Customer', 'Subscriber'], autopct='%1.1f%%')



##### Prepping Data for Analysis

Now that we have the dataset sampled, we still have some legwork necessary to convert our categorical attributes into integer values. Below we walk through this process for the following Attributes:
- start_station_name
- end_station_name
- gender
- DayOfWeek
- TimeOfDay

Once these 5 attributes have been encoded using OneHotEncoding, we have added 79 attributes into our dataset for analysis in our model.

***Start Station Name***

Initially including all start (and end) locations resulted in excessive system resource loading, later during randomized principal component computations, that froze our personal workstations and eventually ended with Python 'MemoryError' messaging. Therefore, due to the extremely large quantity of start stations in our dataset (330 stations), we were required to reduce this dimension down to a manageable size manually. Through trial and error on top frequency stations, we have chosen to reduce this number down to ~ 10% its original number. By identifying the top 20 start stations for Subscribers / Customers separately, we found that there were 9 overlapping stations, producing a final list of 31 stations. While encoding our start_station_name integer columns, we limit the number of columns to these stations identified.

In [ ]:

%%time
    
    #How many Start Stations are there?
print(len(CitiBikeDataSampled_5050["start_station_name"].drop_duplicates()))

    # Top 15 Start Station for Subscriber Users 
startstationsubfreq = pd.DataFrame({'count' : CitiBikeDataSampled_5050[CitiBikeDataSampled_5050["usertype"] == 'Subscriber'].groupby(["start_station_name"]).size()}).reset_index().sort_values('count',ascending = False)
TopSubStartStations = startstationsubfreq.head(20)

del startstationsubfreq

    # Top 15 Start Station for Customer Users 
startstationcustfreq = pd.DataFrame({'count' : CitiBikeDataSampled_5050[CitiBikeDataSampled_5050["usertype"] == 'Customer'].groupby(["start_station_name"]).size()}).reset_index().sort_values('count',ascending = False)
TopCustStartStations = startstationcustfreq.head(20)

del startstationcustfreq

    #Concat Subscribers and Customers
TopStartStations = pd.DataFrame(pd.concat([TopSubStartStations,TopCustStartStations])["start_station_name"].drop_duplicates()).reset_index()    
print(len(TopStartStations))
display(TopStartStations[["start_station_name"]])

del TopStartStations
del TopSubStartStations
del TopCustStartStations

    #Split Start Station Values for 50/50 dataset
AttSplit = pd.get_dummies(CitiBikeDataSampled_5050.start_station_name,prefix='start_station_name')

CitiBikeDataSampled_5050 = pd.concat((CitiBikeDataSampled_5050,AttSplit[["start_station_name_Pershing Square N", "start_station_name_E 17 St & Broadway", "start_station_name_8 Ave & W 31 St", "start_station_name_Lafayette St & E 8 St", "start_station_name_W 21 St & 6 Ave", "start_station_name_8 Ave & W 33 St", "start_station_name_W 20 St & 11 Ave", "start_station_name_Broadway & E 14 St", "start_station_name_Broadway & E 22 St", "start_station_name_W 41 St & 8 Ave", "start_station_name_Cleveland Pl & Spring St", "start_station_name_University Pl & E 14 St", "start_station_name_West St & Chambers St", "start_station_name_E 43 St & Vanderbilt Ave", "start_station_name_Broadway & W 24 St", "start_station_name_Greenwich Ave & 8 Ave", "start_station_name_W 18 St & 6 Ave", "start_station_name_Broadway & W 60 St", "start_station_name_Pershing Square S", "start_station_name_W 33 St & 7 Ave", "start_station_name_Central Park S & 6 Ave", "start_station_name_Centre St & Chambers St", "start_station_name_Grand Army Plaza & Central Park S", "start_station_name_Vesey Pl & River Terrace", "start_station_name_Broadway & W 58 St", "start_station_name_West Thames St", "start_station_name_12 Ave & W 40 St", "start_station_name_9 Ave & W 14 St", "start_station_name_W 14 St & The High Line", "start_station_name_State St", "start_station_name_Broadway & Battery Pl"]]),axis=1) # add back into the dataframe

del AttSplit

***End Station Name***

Similarly, we have an extremely large quantity of end stations in our dataset (330 stations) and including all of them resulted in system crashes during principal component analysis later in our lab. We were required to reduce this dimension down to a manageable size. Through trial and error on top frequency stations, we have chosen to reduce this number down to ~ 10% its original number. By identifying the top 20 end stations for Subscribers / Customers separately, we found that there were 7 overlapping stations, producing a final list of 33 stations. While encoding our end_station_name integer columns, we limit the number of columns to these stations identified.

In [ ]:
%%time
    
    #How many End Stations are there?
print(len(CitiBikeDataSampled_5050["end_station_name"].drop_duplicates()))

    # Top 15 Start Station for Subscriber Users 
endstationsubfreq = pd.DataFrame({'count' : CitiBikeDataSampled_5050[CitiBikeDataSampled_5050["usertype"] == 'Subscriber'].groupby(["end_station_name"]).size()}).reset_index().sort_values('count',ascending = False)
TopSubendStations = endstationsubfreq.head(20)

del endstationsubfreq

    # Top 15 Start Station for Customer Users 
endstationcustfreq = pd.DataFrame({'count' : CitiBikeDataSampled_5050[CitiBikeDataSampled_5050["usertype"] == 'Customer'].groupby(["end_station_name"]).size()}).reset_index().sort_values('count',ascending = False)
TopCustendStations = endstationcustfreq.head(20)

del endstationcustfreq

    #Concat Subscribers and Customers
TopendStations = pd.DataFrame(pd.concat([TopSubendStations,TopCustendStations])["end_station_name"].drop_duplicates()).reset_index()    
print(len(TopendStations))
display(TopendStations[["end_station_name"]])

del TopendStations
del TopSubendStations
del TopCustendStations

    #Split Start Station Values for 50/50 dataset
AttSplit = pd.get_dummies(CitiBikeDataSampled_5050.end_station_name,prefix='end_station_name')

CitiBikeDataSampled_5050 = pd.concat((CitiBikeDataSampled_5050,AttSplit[["end_station_name_E 17 St & Broadway", "end_station_name_Lafayette St & E 8 St", "end_station_name_8 Ave & W 31 St", "end_station_name_W 21 St & 6 Ave", "end_station_name_Pershing Square N", "end_station_name_W 20 St & 11 Ave", "end_station_name_Broadway & E 14 St", "end_station_name_Broadway & E 22 St", "end_station_name_University Pl & E 14 St", "end_station_name_W 41 St & 8 Ave", "end_station_name_West St & Chambers St", "end_station_name_Cleveland Pl & Spring St", "end_station_name_Greenwich Ave & 8 Ave", "end_station_name_E 43 St & Vanderbilt Ave", "end_station_name_Broadway & W 24 St", "end_station_name_W 18 St & 6 Ave", "end_station_name_MacDougal St & Prince St", "end_station_name_Carmine St & 6 Ave", "end_station_name_8 Ave & W 33 St", "end_station_name_2 Ave & E 31 St", "end_station_name_Central Park S & 6 Ave", "end_station_name_Centre St & Chambers St", "end_station_name_Grand Army Plaza & Central Park S", "end_station_name_Broadway & W 60 St", "end_station_name_Broadway & W 58 St", "end_station_name_12 Ave & W 40 St", "end_station_name_Vesey Pl & River Terrace", "end_station_name_W 14 St & The High Line", "end_station_name_9 Ave & W 14 St", "end_station_name_West Thames St", "end_station_name_State St", "end_station_name_Old Fulton St", "end_station_name_South End Ave & Liberty St"]]),axis=1) # add back into the dataframe

del AttSplit

**Gender, DayOfWeek, and TimeOfDay**

The rest of our encoding attributes {Gender, DayOfWeek, and TimeOfDay} have the following value permutations. These permutations will be encoded as individual integer columns as well.

- Gender:    {0 = unknown, 1 = male, 2 = female}
- DayOfWeek: {Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday}
- TimeOfDay: {Morning, Midday, Afternoon, Evening, Night}

In [ ]:
%%time

    #Split gender Values for 50/50 dataset
AttSplit = pd.get_dummies(CitiBikeDataSampled_5050.gender,prefix='gender')
CitiBikeDataSampled_5050 = pd.concat((CitiBikeDataSampled_5050,AttSplit),axis=1) # add back into the dataframe

del AttSplit

    #Split DayOfWeek Values for 50/50 dataset
AttSplit = pd.get_dummies(CitiBikeDataSampled_5050.DayOfWeek,prefix='DayOfWeek')
CitiBikeDataSampled_5050 = pd.concat((CitiBikeDataSampled_5050,AttSplit),axis=1) # add back into the dataframe

del AttSplit

    #Split TimeOfDay Values for 50/50 dataset
AttSplit = pd.get_dummies(CitiBikeDataSampled_5050.TimeOfDay,prefix='TimeOfDay')
CitiBikeDataSampled_5050 = pd.concat((CitiBikeDataSampled_5050,AttSplit),axis=1) # add back into the dataframe

del AttSplit

display(CitiBikeDataSampled_5050.head())


With these encodings complete, our final dataset to cross-validate on test/train datasets would appear to be complete. However, given the large number of attributes now present in our dataset, it would be wise to investigate a means of dimensionality reduction to not only speed up model generation, but to also improve accuracy by removing variable redundancy and correlation.

## Data Preparation Part 2 - Describe the final dataset that is used for classification/regression

##### Dimensionality Reduction using Principal Component Analysis

With our data split evenly among customer and subscriber user types and OneHotEncoding complete, we are ready to consider the new dimensions of our dataset. Because our processed data is comprised of 105 various attributes ranging from weather and distance data to location and user type data across all 500,000 sample observations, and some variables such as weather attributes and even some start and end stations correlate to one another as depicted in the correlation matrix below, we feel it would be wise to reduce our number of attributes considered during model generation. After considering a variety of feature selection techniques, we've opted for dimensionality reduction via principal component analysis (PCA).

In [ ]:
%%time

print(CitiBikeDataSampled_5050.shape)

sns.set(font_scale=0.5)
sns.heatmap(CitiBikeDataSampled_5050.corr(), 
            xticklabels=CitiBikeDataSampled_5050.corr().columns.values,
            yticklabels=CitiBikeDataSampled_5050.corr().columns.values)


By performing PCA, we are able to shrink the number of variables required for prediction purposes, replacing our latest variables derived up to this point with linear combinations of themselves. Each of these linear combinations makes up a component and comprises an eigenvector. When these eigenvectors are ordered by largest eigenvalues, representing the extent of variability explained by each vector, the vectors with largest eigenvalues may be identified as the principal components of the analysis. Selecting only those components which describe the most variance will help us reduce processing times and improve model performance and accuracy by avoiding variable inflation and overfitting associated with high dimensionality.

Since this lab consists of two primary tasks (the first being to classify user types and the second being to predict trip duration), this section outlines two different PCA's, one for each task, in order to identify the appropriate number of components to include during model generation in subsequent sections. The appropriate number of components must be derived separately since user type classification will include trip duration predictors whereas trip duration prediction will require user type predictors. The expectation is that the number of components should be similar for both tasks, but it is not enough to make this assumption alone; both will be reviewed independently.

##### PCA for Classification
Our first objective is to identify the number of components to be used for user type classification. In order to do so, we first exclude redundant and non-value variables up front. Non-value variables include gender, birth year, and age since these data were missing for most Customer user types and were replaced with filler values as discussed in previous sections. We will exclude these since they misrepresent correlation with user type. Remaining attribute data will then be standardized so that all variables are on the same scale given that our explanatory variables are comprised of many different measures.

*(Note: PCA code steps adapted and modified from https://www.analyticsvidhya.com/blog/2016/03/practical-guide-principal-component-analysis-python/)*

In [ ]:
%%time

class_att = CitiBikeDataSampled_5050.drop(['starttime', 'stoptime', 'start_station_id', 'start_station_name', 'end_station_id', 'end_station_name', 'usertype', 'gender', 'gender_0', 'gender_1', 'gender_2', 'birth year', 'Age', 'tripduration', 'DayOfWeek', 'TimeOfDay'], axis=1)
myData = class_att.as_matrix() 

min_max_scaler = MinMaxScaler()
myData_scaled_classification = min_max_scaler.fit_transform(myData)

del myData

After excluding duplicate and non-value attributes and scaling our data, we verify that 89 attributes remain. The maximum number of components to be produced will match this number. For this reason, we will identify 89 to be the number of components produced by our PCA and will review each component's explained variance further to determine the proper number of components to be included later during model generation. Note randomized PCA was chosen in order to use singular value decomposition in our dimensionality reduction efforts due to the large size of our data set. Using full PCA required unacceptable lengths of time to compute.

In [ ]:
%%time
print(myData_scaled_classification.shape)
pca_class = PCA(n_components=89, svd_solver='randomized')

pca_class.fit(myData_scaled_classification)

Below, the resulting components have been ordered by eigenvector value and these values portrayed as ratios of variance explained by each component. In order to identify the principal components to be included during model generation, we review the rate at which explained variance decreases in significance from one principal component to the next. Accompanying these proportion values is a scree plot representing these same values in visual form. By plotting the scree plot, it is easier to judge where this rate of decreasing explained variance occurs. Note the rate of change in explained variance among the first 14 principal components – the change is rather steep through the 14th component. After the 1% drop between components 14 and 15, the rate of decreasing explained variance begins to somewhat flatten out, reducing to a 0.2% change or less.

In [ ]:
#The amount of variance that each PC explains
var= pca_class.explained_variance_ratio_

sns.set(font_scale=1)
plt.plot(range(1,90), var*100, marker = '.', color = 'red', markerfacecolor = 'black')
plt.xlabel('Principal Components')
plt.ylabel('Percentage of Explained Variance')
plt.title('Scree Plot')
plt.axis([0, 90, -0.1, 12])

np.set_printoptions(suppress=True)
print(np.round(var, decimals=4)*100)

By now referring to the cumulative variance values and associated plot below, it may be seen that the cumulative variance arguably begins to plateau around the 14th principal component and that the first 14 components together explain 77.92% of variance in the data set. For this reason, 14 principal components may be selected as being the most appropriate for user type classification modeling given the variables among these data.

In [ ]:
#Cumulative Variance explains
var1=np.cumsum(np.round(pca_class.explained_variance_ratio_, decimals=4)*100)

plt.plot(range(1,90), var1, marker = '.', color = 'green', markerfacecolor = 'black')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance (Sum %)')
plt.title('Cumulative Variance Plot')
plt.axis([0, 90, 10, 101])

print(var1)

In [ ]:
%time pca_classification = PCA(n_components=14, svd_solver='randomized')

##### PCA for Regression

Now that 14 principal components have been identified for Customer/Subscriber classification, we want to determine the correct number of principal components to use when predicting trip duration. For regression, we will need to interchange user type and log trip duration (which will be what we are actually predicting later before back transforming to regular trip duration) inclusion/exclusion. Before including user type, however, OneHotEncoding this variable is required since this has not been done as user type has always been discussed in terms of response up to this point (classification).

We will also be removing end station names for regression since, contextually, CitiBike does not know the end location of a rider at the time the rider checks out a bike. Yes, this information becomes available at checkin... but if the company is to counterbalance the shortage effects of having too many bikes checked out during the same period of time, the model needs to exclude trip end location details. This concept will be discussed further later in this writeup. Again, all attributes are standardized before eigenvectors are computed for reasons stated previously.

In [ ]:
%%time

# Generate usertype dummies for use in regression PCA
AttSplit = pd.get_dummies(CitiBikeDataSampled_5050.usertype,prefix='usertype')
CitiBikeDataSampled_5050 = pd.concat((CitiBikeDataSampled_5050,AttSplit),axis=1) # add back into the dataframe

del AttSplit

# Include usertype but exclude trip duration
reg_att = CitiBikeDataSampled_5050.drop(['starttime', 'stoptime', 'start_station_id', 'start_station_name', 'end_station_id', 'end_station_name', 'usertype','tripdurationLog', 'gender', 'gender_0', 'gender_1', 'gender_2', 'birth year', 'Age', 'tripduration', 'DayOfWeek', 'TimeOfDay', 'LinearDistance'], axis=1)
myData = reg_att.columns.values.tolist() 
myData = [i for i in myData if "end_station_name_" not in i]
reg_att = CitiBikeDataSampled_5050[myData]

min_max_scaler = MinMaxScaler()
myData_scaled_regression = min_max_scaler.fit_transform(reg_att.as_matrix())

del myData

After OneHotEncoding the user type variable, including only appropriate variables, and standardizing, we are left with 56 attributes. Therefore we can expect to produce a total of 56 linear combinations in all.

In [ ]:
%%time

print(myData_scaled_regression.shape)
pca_reg = PCA(n_components=56, svd_solver='randomized')

pca_reg.fit(myData_scaled_regression)

Ordering the resulting eigenvectors by variance proportion explained and plotting the scree plot below reveals dramatic change in explained variance through component 15. The 0.85% change in explained variance between components 15 and 16 flattens out to changes less than 0.33% between components thereafter.

In [ ]:
#The amount of variance that each PC explains
var= pca_reg.explained_variance_ratio_

plt.plot(range(1,57), var*100, marker = '.', color = 'red', markerfacecolor = 'black')
plt.xlabel('Principal Components')
plt.ylabel('Percentage of Explained Variance')
plt.title('Scree Plot')
plt.axis([0, 60, -0.1, 20])

np.set_printoptions(suppress=True)
print(np.round(var, decimals=4)*100)

The cumulative variance values and plot that follow indicate that total variance begins to plateau around the 15th principal component, supporting our previous conclusions. The plot also indicates the first 15 components explain 89.35% of data set variance. Based on these results, we will use 15 principal components for our trip duration prediction models.

In [ ]:
#Cumulative Variance explains
var1=np.cumsum(np.round(pca_reg.explained_variance_ratio_, decimals=4)*100)

plt.plot(range(1,57), var1, marker = '.', color = 'green', markerfacecolor = 'black')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance (Sum %)')
plt.title('Cumulative Variance Plot')
plt.axis([-1, 60, 15, 101])

print(var1)

In [ ]:
%time pca_regression = PCA(n_components=15, svd_solver='randomized')

##### Data Set Summary

At this stage, we've converted our original 30 variables into 107 attributes after creating dummy variables for categorical data such as day of the week, time of day, station names, gender, etc. These 107 attributes and their data types are as follows:

In [ ]:
%%time

data_type = []
for idx, col in enumerate(CitiBikeDataSampled_5050.columns):
    data_type.append(CitiBikeDataSampled_5050.dtypes[idx])

summary_df = {'Attribute Name' : pd.Series(CitiBikeDataSampled_5050.columns, index = range(len(CitiBikeDataSampled_5050.columns))), 'Data Type' : pd.Series(data_type, index = range(len(CitiBikeDataSampled_5050.columns)))}
summary_df = pd.DataFrame(summary_df)
display(summary_df)

del data_type, summary_df

Even with our data cleaned and prepped using OneHotEncoding, there is the innate need to reduce this number of attributes to a more manageable size before classification and regression predictions are made. For this reason, we've performed randomized PCA to compute linear combinations of the data and have chosen to use the first 14 principal components for the *n_components* parameter in our classification PCA and the first 15 principal components for regression PCA, based on explained variance and cumulative variance measures.

Eigenvectors, proportions of explained variance, and cumulative proportions of explained variance are provided for the classification principal components and then again for regression principal components below.

In [ ]:
%%time

# Classification Task Principal Component Breakdown
pca_classification.fit(myData_scaled_classification)
var = pca_classification.explained_variance_
var1 = pca_classification.explained_variance_ratio_
var2 = np.cumsum(np.round(pca_classification.explained_variance_ratio_, decimals=4))
idx = ['PC'+str(i) for i in range(1,15)]

PCA_df = {'...Eigenvector...' : pd.Series(var, index = idx),
          '..Variance Proportion..' : pd.Series(var1, index = idx),
          '.Variance Cumulative Proportion.' : pd.Series(var2, index = idx)}

PCA_df = pd.DataFrame(PCA_df)
print('Classification Task Principal Component Breakdown')
display(PCA_df)

del var, var1, var2, idx
pca_classification = PCA(n_components=14, svd_solver='randomized')

In [ ]:
%%time

# Regression Task Principal Component Breakdown
pca_regression.fit(myData_scaled_regression)
var = pca_regression.explained_variance_
var1 = pca_regression.explained_variance_ratio_
var2 = np.cumsum(np.round(pca_regression.explained_variance_ratio_, decimals=4))
idx = ['PC'+str(i) for i in range(1,16)]

PCA_df = {'...Eigenvector...' : pd.Series(var, index = idx),
          '..Variance Proportion..' : pd.Series(var1, index = idx),
          '.Variance Cumulative Proportion.' : pd.Series(var2, index = idx)}

PCA_df = pd.DataFrame(PCA_df)
print('Regression Task Principal Component Breakdown')
display(PCA_df)

del var, var1, var2, idx
pca_regression = PCA(n_components=15, svd_solver='randomized')

While our discussions revolving data preparation and principal component derivations address our current interests for variable understanding moving forward into modeling and evaluation, there is still more to be understood regarding principal component loadings. Loadings will be described as we discuss the most important attributes for our tasks later in the Modeling and Evaluation 6 section.

### Modeling and Evaluation

##### Choose and explain the evaluation metrics that will be used

### Michael for Eval Metrics
*XXXXXX (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions. XXXXXX*

### Build Test & Train Datasets

#### Classification
With our final encoded dataset complete, and PCA analysis performed to identify our principal components we begin splitting the data into Test vs Train datasets. We have chosen to utilize Stratified KFold Cross Validation for our classification analysis, with 10 folds. This means, that from our original sample size of 500,000, each "fold" will save off approximately 10% as test observations utilizing the rest as training observations all while keeping the ratio of classes equal amongst customers and subscribers. This process will occur through 10 iterations, or folds, to allow us to cross validate our results amongst different test/train combinations. We have utilized a random_state seed equal to the length of the original sampled dataset to ensure reproducible results.  

In [ ]:
    # Create CV Object for StratifiedKFold with 10 Folds, seeded at the length of our sample size
seed = len(CitiBikeDataSampled_5050)

cv = StratifiedKFold(n_splits = 10, random_state = seed)
print(cv)

#### Regression
Alternatively, we have chosen to utilize standard KFold Cross Validation for our regresssion analysis, with 10 folds. This is because our regression target is a continuous attribute, and we have already done stratification of the original dataset in general. We have chosen to shuffle the data before splitting into batches to prevent any potential issues with the order of our dataset. Once again, this means, that from our original sample size of 500,000, data will be randomly shuffled then each "fold" will save off approximately 10% as test observations utilizing the rest as training observations. This process will occur through 10 iterations, or folds, to allow us to cross validate our results amongst different test/train combinations. We continue to utilize the previously established random_state seed equal to the length of the original sampled dataset to ensure reproducible results.  

In [ ]:
    # Create CV Object for KFold with 10 Folds, seeded at the length of our sample size

cvReg = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)
print(cvReg)

### Build/Tune Three Classification Models (usertype predictions)
#### Logistic Regression
cost winner: .01

Accuracy matches .01 and 5.0. We would not expect to need a high cost factor due to the scaler data prior to running our model fit.

In [ ]:
%%time

def lr_explor(cost,  
              ScaledData,
              PCA         = pca_classification,
              Data        = CitiBikeDataSampled_5050,
              cv          = cv,
              seed        = seed):
    
    startTime = datetime.now()
    y = Data['usertype'].values # get the labels we want
    y = np.where(y == 'Subscriber', 1, 0)    
    
    X = ScaledData
    
    lr_clf = LogisticRegression(penalty='l2', C=cost, class_weight=None, random_state=seed) # get object
    
    # setup pipeline to take PCA, then fit a clf model
    clf_pipe = Pipeline(
        [('PCA',PCA),
         ('CLF',lr_clf)]
    )

    accuracy = cross_val_score(clf_pipe, X, y, cv=cv.split(X, y)) # this also can help with parallelism
    MeanAccuracy =  sum(accuracy)/len(accuracy)
    accuracy = np.append(accuracy, MeanAccuracy)
    endTime = datetime.now()
    TotalTime = endTime - startTime
    accuracy = np.append(accuracy, TotalTime)
    return accuracy



In [ ]:
## The Below Code is commented out due to its long run-time. 
##The Rendered HTML Table output has been hardcoded in the next cell block for interpretations 

#%%time
#
#acclist = [] 
#
#cost    = [.01, .05, 1.0, 5.0]
#
#
#for i in range(0,len(cost)):
#    acclist.append(lr_explor(cost       = cost[i],
#                             ScaledData = myData_scaled_classification))
#
#LRcostdf = pd.DataFrame(pd.concat([pd.DataFrame(cost),
#                               pd.DataFrame(acclist)], axis = 1).reindex())
#LRcostdf.columns = ['Cost','Iteration 0', 'Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6', 'Iteration 7', 'Iteration 8', 'Iteration 9', 'MeanAccuracy', 'RunTime']
#display(LRcostdf)

<div class="output_subarea output_html rendered_html"><div>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Cost</th>
      <th>Iteration 0</th>
      <th>Iteration 1</th>
      <th>Iteration 2</th>
      <th>Iteration 3</th>
      <th>Iteration 4</th>
      <th>Iteration 5</th>
      <th>Iteration 6</th>
      <th>Iteration 7</th>
      <th>Iteration 8</th>
      <th>Iteration 9</th>
      <th>MeanAccuracy</th>
      <th>RunTime</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.01</td>
      <td>0.68128</td>
      <td>0.68198</td>
      <td>0.68054</td>
      <td>0.68190</td>
      <td>0.68584</td>
      <td>0.67858</td>
      <td>0.68522</td>
      <td>0.67878</td>
      <td>0.68100</td>
      <td>0.68374</td>
      <td>0.681886</td>
      <td>00:00:55.301026</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.05</td>
      <td>0.68122</td>
      <td>0.68212</td>
      <td>0.68052</td>
      <td>0.68208</td>
      <td>0.68588</td>
      <td>0.67842</td>
      <td>0.68508</td>
      <td>0.67870</td>
      <td>0.68080</td>
      <td>0.68372</td>
      <td>0.681854</td>
      <td>00:00:54.031372</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1.00</td>
      <td>0.68136</td>
      <td>0.68208</td>
      <td>0.68050</td>
      <td>0.68208</td>
      <td>0.68586</td>
      <td>0.67848</td>
      <td>0.68504</td>
      <td>0.67882</td>
      <td>0.68080</td>
      <td>0.68374</td>
      <td>0.681876</td>
      <td>00:00:53.148202</td>
    </tr>
    <tr>
      <th>3</th>
      <td>5.00</td>
      <td>0.68138</td>
      <td>0.68202</td>
      <td>0.68048</td>
      <td>0.68210</td>
      <td>0.68586</td>
      <td>0.67846</td>
      <td>0.68508</td>
      <td>0.67876</td>
      <td>0.68082</td>
      <td>0.68376</td>
      <td>0.681872</td>
      <td>00:00:52.870310</td>
    </tr>
  </tbody>
</table>
</div></div>

#### Random Forest

winner: Last Iteration (12)
	max_depth	max_features	min_samples_leaf	min_samples_split	n_estimators
	1000.0	    14	            25	                50	                15

In [ ]:
%%time


def rfc_explor(ScaledData,
               n_estimators,
               max_features,
               max_depth, 
               min_samples_split,
               min_samples_leaf, 
               PCA         = pca_classification,
               Data        = CitiBikeDataSampled_5050,
               cv          = cv,
               seed        = seed):
    startTime = datetime.now()
    y = Data['usertype'].values # get the labels we want
    y = np.where(y == 'Subscriber', 1, 0)    
    
    X = ScaledData
    
    rfc_clf = RandomForestClassifier(n_estimators=n_estimators, max_features = max_features, max_depth=max_depth, min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf, n_jobs=-1, random_state = seed) # get object
    
    # setup pipeline to take PCA, then fit a clf model
    clf_pipe = Pipeline(
        [('PCA',PCA),
         ('CLF',rfc_clf)]
    )

    accuracy = cross_val_score(clf_pipe, X, y, cv=cv.split(X, y)) # this also can help with parallelism
    MeanAccuracy =  sum(accuracy)/len(accuracy)
    accuracy = np.append(accuracy, MeanAccuracy)
    endTime = datetime.now()
    TotalTime = endTime - startTime
    accuracy = np.append(accuracy, TotalTime)
    
    #print(TotalTime)
    #print(accuracy)
    
    return accuracy


In [ ]:

## The Below Code is commented out due to its long run-time. 
##The Rendered HTML Table output has been hardcoded in the next cell block for interpretations 


#%%time
#
#acclist = [] 
#
#n_estimators       =  [10    , 10     , 10    , 10    , 10    , 10    , 10    , 10    , 10    , 10    , 10  , 5    , 15   ]  
#max_features       =  ['auto', 'auto' , 'auto', 'auto', 'auto', 'auto', 'auto', 14    , 14    , 14    , 14  , 14   , 14   ] 
#max_depth          =  [None  , None   , None  , None  , None  , None  , None  , None  , 1000  , 500   , 100 , 1000 , 1000 ] 
#min_samples_split  =  [2     , 8      , 12    , 16    , 20    , 50    , 80    , 50    , 50    , 50    , 50  , 50   , 50   ] 
#min_samples_leaf   =  [1     , 4      , 6     , 8     , 10    , 25    , 40    , 25    , 25    , 25    , 25  , 25   , 25   ]
#
#for i in range(0,len(n_estimators)):
#    acclist.append(rfc_explor(ScaledData        = myData_scaled_classification,
#                              n_estimators      = n_estimators[i],
#                              max_features      = max_features[i],
#                              max_depth         = max_depth[i],
#                              min_samples_split = min_samples_split[i],
#                              min_samples_leaf  = min_samples_leaf[i]
#                             )
#                  )
#
#rfcdf = pd.DataFrame(pd.concat([pd.DataFrame({
#                                                "n_estimators": n_estimators,          
#                                                "max_features": max_features,         
#                                                "max_depth": max_depth,        
#                                                "min_samples_split": min_samples_split,
#                                                "min_samples_leaf": min_samples_leaf   
#                                              }),
#                               pd.DataFrame(acclist)], axis = 1).reindex())
#rfcdf.columns = ['max_depth', 'max_features', 'min_samples_leaf','min_samples_split', 'n_estimators', 'Iteration 0', 'Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6', 'Iteration 7', 'Iteration 8', 'Iteration 9', 'MeanAccuracy', 'RunTime']
#display(rfcdf)




<div>
<table border=\1\ class=\dataframe\>
 <thead>
   <tr style=\text-align: right;\>
     <th></th>
     <th>max_depth</th>
     <th>max_features</th>
     <th>min_samples_leaf</th>
     <th>min_samples_split</th>
     <th>n_estimators</th>
     <th>Iteration 0</th>
     <th>Iteration 1</th>
     <th>Iteration 2</th>
     <th>Iteration 3</th>
     <th>Iteration 4</th>
     <th>Iteration 5</th>
     <th>Iteration 6</th>
     <th>Iteration 7</th>
     <th>Iteration 8</th>
     <th>Iteration 9</th>
     <th>MeanAccuracy</th>
     <th>RunTime</th>
   </tr>
 </thead>
 <tbody>
   <tr>
     <th>0</th>
     <td>NaN</td>
     <td>auto</td>
     <td>1</td>
     <td>2</td>
     <td>10</td>
     <td>0.70206</td>
     <td>0.70432</td>
     <td>0.70548</td>
     <td>0.70046</td>
     <td>0.70300</td>
     <td>0.69678</td>
     <td>0.70606</td>
     <td>0.70146</td>
     <td>0.70256</td>
     <td>0.70450</td>
     <td>0.702668</td>
     <td>00:02:22.858960</td>
   </tr>
   <tr>
     <th>1</th>
     <td>NaN</td>
     <td>auto</td>
     <td>4</td>
     <td>8</td>
     <td>10</td>
     <td>0.71310</td>
     <td>0.71480</td>
     <td>0.71486</td>
     <td>0.71398</td>
     <td>0.71414</td>
     <td>0.70878</td>
     <td>0.71756</td>
     <td>0.71150</td>
     <td>0.71694</td>
     <td>0.71564</td>
     <td>0.714130</td>
     <td>00:02:19.138254</td>
   </tr>
   <tr>
     <th>2</th>
     <td>NaN</td>
     <td>auto</td>
     <td>6</td>
     <td>12</td>
     <td>10</td>
     <td>0.71576</td>
     <td>0.71660</td>
     <td>0.72088</td>
     <td>0.71724</td>
     <td>0.71766</td>
     <td>0.71410</td>
     <td>0.71964</td>
     <td>0.71730</td>
     <td>0.71990</td>
     <td>0.71808</td>
     <td>0.717716</td>
     <td>00:02:18.481832</td>
   </tr>
   <tr>
     <th>3</th>
     <td>NaN</td>
     <td>auto</td>
     <td>8</td>
     <td>16</td>
     <td>10</td>
     <td>0.72052</td>
     <td>0.72110</td>
     <td>0.72216</td>
     <td>0.71840</td>
     <td>0.71940</td>
     <td>0.71478</td>
     <td>0.72130</td>
     <td>0.71560</td>
     <td>0.71952</td>
     <td>0.71852</td>
     <td>0.719130</td>
     <td>00:02:17.359507</td>
   </tr>
   <tr>
     <th>4</th>
     <td>NaN</td>
     <td>auto</td>
     <td>10</td>
     <td>20</td>
     <td>10</td>
     <td>0.71878</td>
     <td>0.72088</td>
     <td>0.72286</td>
     <td>0.71678</td>
     <td>0.72022</td>
     <td>0.71692</td>
     <td>0.72426</td>
     <td>0.71810</td>
     <td>0.72134</td>
     <td>0.72066</td>
     <td>0.720080</td>
     <td>00:02:15.995898</td>
   </tr>
   <tr>
     <th>5</th>
     <td>NaN</td>
     <td>auto</td>
     <td>25</td>
     <td>50</td>
     <td>10</td>
     <td>0.72098</td>
     <td>0.72240</td>
     <td>0.72164</td>
     <td>0.71710</td>
     <td>0.72258</td>
     <td>0.71856</td>
     <td>0.72334</td>
     <td>0.71908</td>
     <td>0.72254</td>
     <td>0.72320</td>
     <td>0.721142</td>
     <td>00:02:11.538186</td>
   </tr>
   <tr>
     <th>6</th>
     <td>NaN</td>
     <td>auto</td>
     <td>40</td>
     <td>80</td>
     <td>10</td>
     <td>0.71874</td>
     <td>0.71902</td>
     <td>0.72102</td>
     <td>0.71716</td>
     <td>0.72050</td>
     <td>0.71686</td>
     <td>0.72410</td>
     <td>0.71760</td>
     <td>0.72158</td>
     <td>0.72104</td>
     <td>0.719762</td>
     <td>00:02:06.851562</td>
   </tr>
   <tr>
     <th>7</th>
     <td>NaN</td>
     <td>14</td>
     <td>25</td>
     <td>50</td>
     <td>10</td>
     <td>0.72230</td>
     <td>0.72428</td>
     <td>0.72382</td>
     <td>0.71876</td>
     <td>0.72684</td>
     <td>0.71958</td>
     <td>0.72524</td>
     <td>0.72160</td>
     <td>0.72436</td>
     <td>0.72328</td>
     <td>0.723006</td>
     <td>00:06:37.091719</td>
   </tr>
   <tr>
     <th>8</th>
     <td>1000.0</td>
     <td>14</td>
     <td>25</td>
     <td>50</td>
     <td>10</td>
     <td>0.72260</td>
     <td>0.72488</td>
     <td>0.72530</td>
     <td>0.72070</td>
     <td>0.72446</td>
     <td>0.71846</td>
     <td>0.72462</td>
     <td>0.72124</td>
     <td>0.72546</td>
     <td>0.72346</td>
     <td>0.723118</td>
     <td>00:06:33.179886</td>
   </tr>
   <tr>
     <th>9</th>
     <td>500.0</td>
     <td>14</td>
     <td>25</td>
     <td>50</td>
     <td>10</td>
     <td>0.72228</td>
     <td>0.72430</td>
     <td>0.72430</td>
     <td>0.71970</td>
     <td>0.72520</td>
     <td>0.71920</td>
     <td>0.72702</td>
     <td>0.72076</td>
     <td>0.72326</td>
     <td>0.72404</td>
     <td>0.723006</td>
     <td>00:06:35.014423</td>
   </tr>
   <tr>
     <th>10</th>
     <td>100.0</td>
     <td>14</td>
     <td>25</td>
     <td>50</td>
     <td>10</td>
     <td>0.72234</td>
     <td>0.72360</td>
     <td>0.72392</td>
     <td>0.71976</td>
     <td>0.72480</td>
     <td>0.71958</td>
     <td>0.72520</td>
     <td>0.72080</td>
     <td>0.72534</td>
     <td>0.72464</td>
     <td>0.722998</td>
     <td>00:06:33.892724</td>
   </tr>
   <tr>
     <th>11</th>
     <td>1000.0</td>
     <td>14</td>
     <td>25</td>
     <td>50</td>
     <td>5</td>
     <td>0.71642</td>
     <td>0.71852</td>
     <td>0.71884</td>
     <td>0.71458</td>
     <td>0.71964</td>
     <td>0.71342</td>
     <td>0.71864</td>
     <td>0.71534</td>
     <td>0.71868</td>
     <td>0.71980</td>
     <td>0.717388</td>
     <td>00:04:21.575723</td>
   </tr>
   <tr>
     <th>12</th>
     <td>1000.0</td>
     <td>14</td>
     <td>25</td>
     <td>50</td>
     <td>15</td>
     <td>0.72432</td>
     <td>0.72640</td>
     <td>0.72618</td>
     <td>0.72126</td>
     <td>0.72712</td>
     <td>0.72128</td>
     <td>0.72728</td>
     <td>0.72448</td>
     <td>0.72708</td>
     <td>0.72652</td>
     <td>0.725192</td>
     <td>00:08:59.111516</td>
   </tr>
 </tbody>
</table>
</div>
Wall time: 55min 32s

#### KNN

Winner: 
    algorithm	leaf_size	n_neighbors
    kd_tree	    50	        150

Note: we lose 00.04% accuracy between KD_tree and ball_tree, but we gain 7min 25 secs in runtime - pushing KD_tree as our top choice. 

In [ ]:
%%time


def knn_explor(ScaledData,
               n_neighbors,
               algorithm ,
               leaf_size,
               PCA         = pca_classification,
               Data        = CitiBikeDataSampled_5050,
               cv          = cv,
               seed        = seed):
    startTime = datetime.now()
    y = Data['usertype'].values # get the labels we want
    y = np.where(y == 'Subscriber', 1, 0)    
    
    X = ScaledData
    
    knn_clf = KNeighborsClassifier(n_neighbors = n_neighbors, algorithm = algorithm, leaf_size = leaf_size, n_jobs=-1) # get object
    
    # setup pipeline to take PCA, then fit a clf model
    clf_pipe = Pipeline(
        [('PCA',PCA),
         ('CLF',knn_clf)]
    )

    accuracy = cross_val_score(clf_pipe, X, y, cv=cv.split(X, y)) # this also can help with parallelism
    MeanAccuracy =  sum(accuracy)/len(accuracy)
    accuracy = np.append(accuracy, MeanAccuracy)
    endTime = datetime.now()
    TotalTime = endTime - startTime
    accuracy = np.append(accuracy, TotalTime)
    
    #print(TotalTime)
    #print(accuracy)
    
    return accuracy


In [ ]:
## The Below Code is commented out due to its long run-time. 
##The Rendered HTML Table output has been hardcoded in the next cell block for interpretations 


#%%time
#
#acclist = [] 
#
#n_neighbors =  [5          , 5          , 5          , 5          , 5            , 10          , 15          , 20          , 50          , 100         , 150         , 200         ] 
#algorithm   =  ['ball_tree'     , 'ball_tree'     , 'ball_tree'     , 'ball_tree'     , 'ball_tree'       , 'ball_tree'      , 'ball_tree'      , 'ball_tree'      , 'ball_tree'      , 'ball_tree'      , 'ball_tree'      , 'ball_tree'      ] 
#leaf_size   =  [30         , 15         , 50         , 75         , 50           , 50          , 50          , 50          , 50          , 50          , 50          , 50          ] 
#
#
#
#for i in range(0,len(n_neighbors)):
#    acclist.append(knn_explor(ScaledData  = myData_scaled_classification,
#                              n_neighbors = n_neighbors[i],
#                              algorithm   = algorithm[i],
#                              leaf_size   = leaf_size[i]
#                             )
#                  )
#
#rfcdf = pd.DataFrame(pd.concat([pd.DataFrame({
#                                                "n_neighbors": n_neighbors,          
#                                                "algorithm": algorithm,         
#                                                "leaf_size": leaf_size  
#                                              }),
#                               pd.DataFrame(acclist)], axis = 1).reindex())
#rfcdf.columns = ['algorithm', 'leaf_size','n_neighbors', 'Iteration 0', 'Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6', 'Iteration 7', 'Iteration 8', 'Iteration 9', 'MeanAccuracy', 'RunTime']
#display(rfcdf)
#
#
#
#
#acclist = [] 
#
#n_neighbors =  [5          , 5          , 5          , 5          , 5            , 10          , 15          , 20          , 50          , 100         , 150         , 200         ] 
#algorithm   =  ['kd_tree'     , 'kd_tree'     , 'kd_tree'     , 'kd_tree'     , 'kd_tree'       , 'kd_tree'      , 'kd_tree'      , 'kd_tree'      , 'kd_tree'      , 'kd_tree'      , 'kd_tree'      , 'kd_tree'      ] 
#leaf_size   =  [30         , 15         , 50         , 75         , 50           , 50          , 50          , 50          , 50          , 50          , 50          , 50          ] 
#
#
#
#for i in range(0,len(n_neighbors)):
#    acclist.append(knn_explor(ScaledData  = myData_scaled_classification,
#                              n_neighbors = n_neighbors[i],
#                              algorithm   = algorithm[i],
#                              leaf_size   = leaf_size[i]
#                             )
#                  )
#
#rfcdf = pd.DataFrame(pd.concat([pd.DataFrame({
#                                                "n_neighbors": n_neighbors,          
#                                                "algorithm": algorithm,         
#                                                "leaf_size": leaf_size  
#                                              }),
#                               pd.DataFrame(acclist)], axis = 1).reindex())
#rfcdf.columns = ['algorithm', 'leaf_size','n_neighbors', 'Iteration 0', 'Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6', 'Iteration 7', 'Iteration 8', 'Iteration 9', 'MeanAccuracy', 'RunTime']
#display(rfcdf)




<div class="output_subarea output_html rendered_html"><div>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>algorithm</th>
      <th>leaf_size</th>
      <th>n_neighbors</th>
      <th>Iteration 0</th>
      <th>Iteration 1</th>
      <th>Iteration 2</th>
      <th>Iteration 3</th>
      <th>Iteration 4</th>
      <th>Iteration 5</th>
      <th>Iteration 6</th>
      <th>Iteration 7</th>
      <th>Iteration 8</th>
      <th>Iteration 9</th>
      <th>MeanAccuracy</th>
      <th>RunTime</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ball_tree</td>
      <td>30</td>
      <td>5</td>
      <td>0.66732</td>
      <td>0.66576</td>
      <td>0.66590</td>
      <td>0.66382</td>
      <td>0.66776</td>
      <td>0.66150</td>
      <td>0.66576</td>
      <td>0.66194</td>
      <td>0.66566</td>
      <td>0.66724</td>
      <td>0.665266</td>
      <td>00:06:53.743385</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ball_tree</td>
      <td>15</td>
      <td>5</td>
      <td>0.66722</td>
      <td>0.66580</td>
      <td>0.66586</td>
      <td>0.66406</td>
      <td>0.66784</td>
      <td>0.66154</td>
      <td>0.66562</td>
      <td>0.66188</td>
      <td>0.66534</td>
      <td>0.66700</td>
      <td>0.665216</td>
      <td>00:05:40.484082</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>5</td>
      <td>0.66716</td>
      <td>0.66572</td>
      <td>0.66606</td>
      <td>0.66422</td>
      <td>0.66790</td>
      <td>0.66162</td>
      <td>0.66588</td>
      <td>0.66182</td>
      <td>0.66572</td>
      <td>0.66718</td>
      <td>0.665328</td>
      <td>00:06:54.453049</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ball_tree</td>
      <td>75</td>
      <td>5</td>
      <td>0.66748</td>
      <td>0.66598</td>
      <td>0.66572</td>
      <td>0.66386</td>
      <td>0.66786</td>
      <td>0.66178</td>
      <td>0.66580</td>
      <td>0.66192</td>
      <td>0.66540</td>
      <td>0.66708</td>
      <td>0.665288</td>
      <td>00:09:07.276291</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>5</td>
      <td>0.66718</td>
      <td>0.66592</td>
      <td>0.66586</td>
      <td>0.66398</td>
      <td>0.66806</td>
      <td>0.66152</td>
      <td>0.66576</td>
      <td>0.66190</td>
      <td>0.66542</td>
      <td>0.66698</td>
      <td>0.665258</td>
      <td>00:06:54.266485</td>
    </tr>
    <tr>
      <th>5</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>10</td>
      <td>0.68034</td>
      <td>0.67936</td>
      <td>0.67816</td>
      <td>0.67758</td>
      <td>0.68036</td>
      <td>0.67388</td>
      <td>0.67840</td>
      <td>0.67480</td>
      <td>0.68068</td>
      <td>0.67866</td>
      <td>0.678222</td>
      <td>00:07:03.431483</td>
    </tr>
    <tr>
      <th>6</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>15</td>
      <td>0.68648</td>
      <td>0.68584</td>
      <td>0.68636</td>
      <td>0.68364</td>
      <td>0.68574</td>
      <td>0.68160</td>
      <td>0.68682</td>
      <td>0.68154</td>
      <td>0.68738</td>
      <td>0.68630</td>
      <td>0.685170</td>
      <td>00:07:14.175179</td>
    </tr>
    <tr>
      <th>7</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>20</td>
      <td>0.68922</td>
      <td>0.68816</td>
      <td>0.68828</td>
      <td>0.68702</td>
      <td>0.68972</td>
      <td>0.68486</td>
      <td>0.69022</td>
      <td>0.68466</td>
      <td>0.69014</td>
      <td>0.68880</td>
      <td>0.688108</td>
      <td>00:07:45.081930</td>
    </tr>
    <tr>
      <th>8</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>50</td>
      <td>0.69556</td>
      <td>0.69452</td>
      <td>0.69446</td>
      <td>0.69290</td>
      <td>0.69572</td>
      <td>0.69068</td>
      <td>0.69476</td>
      <td>0.69210</td>
      <td>0.69590</td>
      <td>0.69468</td>
      <td>0.694128</td>
      <td>00:08:10.106731</td>
    </tr>
    <tr>
      <th>9</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>100</td>
      <td>0.69504</td>
      <td>0.69628</td>
      <td>0.69582</td>
      <td>0.69370</td>
      <td>0.69856</td>
      <td>0.69170</td>
      <td>0.69632</td>
      <td>0.69318</td>
      <td>0.69630</td>
      <td>0.69670</td>
      <td>0.695360</td>
      <td>00:08:54.870863</td>
    </tr>
    <tr>
      <th>10</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>150</td>
      <td>0.69618</td>
      <td>0.69692</td>
      <td>0.69596</td>
      <td>0.69322</td>
      <td>0.69690</td>
      <td>0.69150</td>
      <td>0.69764</td>
      <td>0.69344</td>
      <td>0.69672</td>
      <td>0.69680</td>
      <td>0.695528</td>
      <td>00:09:27.100008</td>
    </tr>
    <tr>
      <th>11</th>
      <td>ball_tree</td>
      <td>50</td>
      <td>200</td>
      <td>0.69592</td>
      <td>0.69678</td>
      <td>0.69454</td>
      <td>0.69374</td>
      <td>0.69716</td>
      <td>0.69184</td>
      <td>0.69722</td>
      <td>0.69380</td>
      <td>0.69678</td>
      <td>0.69588</td>
      <td>0.695366</td>
      <td>00:09:52.138668</td>
    </tr>
  </tbody>
</table>
Wall time: 1h 33min 57s

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>algorithm</th>
      <th>leaf_size</th>
      <th>n_neighbors</th>
      <th>Iteration 0</th>
      <th>Iteration 1</th>
      <th>Iteration 2</th>
      <th>Iteration 3</th>
      <th>Iteration 4</th>
      <th>Iteration 5</th>
      <th>Iteration 6</th>
      <th>Iteration 7</th>
      <th>Iteration 8</th>
      <th>Iteration 9</th>
      <th>MeanAccuracy</th>
      <th>RunTime</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>kd_tree</td>
      <td>30</td>
      <td>5</td>
      <td>0.66726</td>
      <td>0.66588</td>
      <td>0.66568</td>
      <td>0.66414</td>
      <td>0.66804</td>
      <td>0.66150</td>
      <td>0.66570</td>
      <td>0.66168</td>
      <td>0.66564</td>
      <td>0.66714</td>
      <td>0.665266</td>
      <td>00:01:13.676624</td>
    </tr>
    <tr>
      <th>1</th>
      <td>kd_tree</td>
      <td>15</td>
      <td>5</td>
      <td>0.66732</td>
      <td>0.66598</td>
      <td>0.66574</td>
      <td>0.66376</td>
      <td>0.66764</td>
      <td>0.66182</td>
      <td>0.66572</td>
      <td>0.66194</td>
      <td>0.66548</td>
      <td>0.66706</td>
      <td>0.665246</td>
      <td>00:01:10.890032</td>
    </tr>
    <tr>
      <th>2</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>5</td>
      <td>0.66726</td>
      <td>0.66576</td>
      <td>0.66590</td>
      <td>0.66402</td>
      <td>0.66802</td>
      <td>0.66158</td>
      <td>0.66562</td>
      <td>0.66180</td>
      <td>0.66518</td>
      <td>0.66734</td>
      <td>0.665248</td>
      <td>00:01:12.039968</td>
    </tr>
    <tr>
      <th>3</th>
      <td>kd_tree</td>
      <td>75</td>
      <td>5</td>
      <td>0.66738</td>
      <td>0.66600</td>
      <td>0.66600</td>
      <td>0.66366</td>
      <td>0.66794</td>
      <td>0.66162</td>
      <td>0.66586</td>
      <td>0.66160</td>
      <td>0.66542</td>
      <td>0.66720</td>
      <td>0.665268</td>
      <td>00:01:17.640469</td>
    </tr>
    <tr>
      <th>4</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>5</td>
      <td>0.66718</td>
      <td>0.66580</td>
      <td>0.66588</td>
      <td>0.66394</td>
      <td>0.66802</td>
      <td>0.66152</td>
      <td>0.66578</td>
      <td>0.66184</td>
      <td>0.66560</td>
      <td>0.66700</td>
      <td>0.665256</td>
      <td>00:01:17.354629</td>
    </tr>
    <tr>
      <th>5</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>10</td>
      <td>0.68052</td>
      <td>0.67930</td>
      <td>0.67830</td>
      <td>0.67748</td>
      <td>0.68034</td>
      <td>0.67394</td>
      <td>0.67810</td>
      <td>0.67488</td>
      <td>0.68094</td>
      <td>0.67860</td>
      <td>0.678240</td>
      <td>00:01:19.506095</td>
    </tr>
    <tr>
      <th>6</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>15</td>
      <td>0.68658</td>
      <td>0.68566</td>
      <td>0.68624</td>
      <td>0.68360</td>
      <td>0.68588</td>
      <td>0.68146</td>
      <td>0.68650</td>
      <td>0.68156</td>
      <td>0.68738</td>
      <td>0.68594</td>
      <td>0.685080</td>
      <td>00:01:23.414543</td>
    </tr>
    <tr>
      <th>7</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>20</td>
      <td>0.68918</td>
      <td>0.68810</td>
      <td>0.68830</td>
      <td>0.68724</td>
      <td>0.68992</td>
      <td>0.68492</td>
      <td>0.69012</td>
      <td>0.68480</td>
      <td>0.69018</td>
      <td>0.68854</td>
      <td>0.688130</td>
      <td>00:01:22.808268</td>
    </tr>
    <tr>
      <th>8</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>50</td>
      <td>0.69550</td>
      <td>0.69440</td>
      <td>0.69404</td>
      <td>0.69292</td>
      <td>0.69560</td>
      <td>0.69072</td>
      <td>0.69500</td>
      <td>0.69222</td>
      <td>0.69570</td>
      <td>0.69454</td>
      <td>0.694064</td>
      <td>00:01:34.127069</td>
    </tr>
    <tr>
      <th>9</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>100</td>
      <td>0.69496</td>
      <td>0.69624</td>
      <td>0.69570</td>
      <td>0.69372</td>
      <td>0.69846</td>
      <td>0.69174</td>
      <td>0.69630</td>
      <td>0.69314</td>
      <td>0.69638</td>
      <td>0.69678</td>
      <td>0.695342</td>
      <td>00:01:49.578879</td>
    </tr>
    <tr>
      <th>10</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>150</td>
      <td>0.69610</td>
      <td>0.69696</td>
      <td>0.69588</td>
      <td>0.69322</td>
      <td>0.69702</td>
      <td>0.69148</td>
      <td>0.69770</td>
      <td>0.69356</td>
      <td>0.69670</td>
      <td>0.69662</td>
      <td>0.695524</td>
      <td>00:02:02.480648</td>
    </tr>
    <tr>
      <th>11</th>
      <td>kd_tree</td>
      <td>50</td>
      <td>200</td>
      <td>0.69592</td>
      <td>0.69694</td>
      <td>0.69454</td>
      <td>0.69374</td>
      <td>0.69726</td>
      <td>0.69158</td>
      <td>0.69724</td>
      <td>0.69384</td>
      <td>0.69672</td>
      <td>0.69586</td>
      <td>0.695364</td>
      <td>00:02:11.007620</td>
    </tr>
  </tbody>
</table>
Wall time: 17min 54s
</div>

### Build/Tune Three Regression Models (TripDurationLog predictions)
#### KNN Regression

In [ ]:
%%time


def knnr_explor(ScaledData,
               n_neighbors,
               algorithm ,
               leaf_size,
               PCA         = pca_regression,
               Data        = CitiBikeDataSampled_5050,
               cv          = cvReg,
               seed        = seed):
    startTime = datetime.now()
    y = Data['tripdurationLog'].values # get the labels we want
    
    X = ScaledData
    
    knnr_clf = KNeighborsRegressor(n_neighbors = n_neighbors, algorithm = algorithm, leaf_size = leaf_size, n_jobs=-1) # get object
    
    # setup pipeline to take PCA, then fit a clf model
    clf_pipe = Pipeline(
        [('PCA',PCA),
         ('CLF',knnr_clf)]
    )

    accuracy = cross_val_score(clf_pipe, X, y, scoring = 'neg_mean_squared_error', cv=cv.split(X, y)) # this also can help with parallelism
    accuracy = accuracy * -1
    MeanAccuracy =  sum(accuracy)/len(accuracy)
    accuracy = np.append(accuracy, MeanAccuracy)
    endTime = datetime.now()
    TotalTime = endTime - startTime
    accuracy = np.append(accuracy, TotalTime)
    
    #print(TotalTime)
    #print(accuracy)
    
    return accuracy


In [ ]:
%%time

acclist = [] 

n_neighbors =  [5          , 5          , 5          , 5          , 5            , 10          , 15          , 20          , 50          , 100         , 150         , 200         ] 
algorithm   =  'ball_tree'
leaf_size   =  [30         , 15         , 50         , 75         , 50           , 50          , 50          , 50          , 50          , 50          , 50          , 50          ] 



for i in range(0,len(n_neighbors)):
    acclist.append(knnr_explor(ScaledData  = myData_scaled_regression,
                               n_neighbors = n_neighbors[i],
                               algorithm   = algorithm,
                               leaf_size   = leaf_size[i]
                             )
                  )

knnrdf = pd.DataFrame(pd.concat([pd.DataFrame({
                                                "n_neighbors": n_neighbors,          
                                                "algorithm": algorithm,         
                                                "leaf_size": leaf_size  
                                              }),
                               pd.DataFrame(acclist)], axis = 1).reindex())
knnrdf.columns = ['algorithm', 'leaf_size','n_neighbors', 'Iteration 0', 'Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6', 'Iteration 7', 'Iteration 8', 'Iteration 9', 'MeanSquaredError', 'RunTime']
display(knnrdf)


acclist = []

n_neighbors =  [5          , 5          , 5          , 5          , 5            , 10          , 15          , 20          , 50          , 100         , 150         , 200         ] 
algorithm   =  'kd_tree'
leaf_size   =  [30         , 15         , 50         , 75         , 50           , 50          , 50          , 50          , 50          , 50          , 50          , 50          ] 

for i in range(0,len(n_neighbors)):
    acclist.append(knnr_explor(ScaledData  = myData_scaled_regression,
                               n_neighbors = n_neighbors[i],
                               algorithm   = algorithm,
                               leaf_size   = leaf_size[i]
                             )
                  )

knnrdf = pd.DataFrame(pd.concat([pd.DataFrame({
                                                "n_neighbors": n_neighbors,          
                                                "algorithm": algorithm,         
                                                "leaf_size": leaf_size  
                                              }),
                               pd.DataFrame(acclist)], axis = 1).reindex())
knnrdf.columns = ['algorithm', 'leaf_size','n_neighbors', 'Iteration 0', 'Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6', 'Iteration 7', 'Iteration 8', 'Iteration 9', 'MeanSquaredError', 'RunTime']
display(rfcdf)

### DecisionTree Regression

In [ ]:
%%time


def dtr_explor(ScaledData,
                splitter,
                max_features ,
                max_depth,
                min_samples_split,
                min_samples_leaf,
                PCA         = pca_regression,
                Data        = CitiBikeDataSampled_5050,
                cv          = cvReg,
                seed        = seed):
    startTime = datetime.now()
    y = Data['tripdurationLog'].values # get the labels we want
    
    X = ScaledData
    
    dtr_clf = DecisionTreeRegressor(splitter           = splitter, 
                                    max_features       = max_features, 
                                    max_depth          = max_depth, 
                                    min_samples_split  = min_samples_split, 
                                    min_samples_leaf   = min_samples_leaf, 
                                    random_state       = seed, 
                                    n_jobs             = -1) # get object
    
    # setup pipeline to take PCA, then fit a clf model
    clf_pipe = Pipeline(
        [('PCA',PCA),
         ('CLF',dtr_clf)]
    )

    accuracy = cross_val_score(clf_pipe, X, y, scoring = 'neg_mean_squared_error', cv=cv.split(X, y)) # this also can help with parallelism
    accuracy = accuracy * -1
    MeanAccuracy =  sum(accuracy)/len(accuracy)
    accuracy = np.append(accuracy, MeanAccuracy)
    endTime = datetime.now()
    TotalTime = endTime - startTime
    accuracy = np.append(accuracy, TotalTime)
    
    #print(TotalTime)
    #print(accuracy)
    
    return accuracy


In [ ]:
%%time

acclist = [] 

splitter           =  'best' 
max_features       =  ['auto', 'auto' , 'auto', 'auto', 'auto', 'auto', 'auto', 14    , 14    , 14    , 14  , 14   , 14   ] 
max_depth          =  [None  , None   , None  , None  , None  , None  , None  , None  , 1000  , 500   , 100 , 1000 , 1000 ] 
min_samples_split  =  [2     , 8      , 12    , 16    , 20    , 50    , 80    , 50    , 50    , 50    , 50  , 50   , 50   ] 
min_samples_leaf   =  [1     , 4      , 6     , 8     , 10    , 25    , 40    , 25    , 25    , 25    , 25  , 25   , 25   ]



for i in range(0,len(max_features)):
    acclist.append(knnr_explor(ScaledData         = myData_scaled_regression,
                               splitter           = splitter, 
                               max_features       = max_features[i], 
                               max_depth          = max_depth[i], 
                               min_samples_split  = min_samples_split[i],
                               min_samples_leaf   = min_samples_leaf[i],
                              )
                  )

knnrdf = pd.DataFrame(pd.concat([pd.DataFrame({
                                                "splitter         ": splitter         ,
                                                "max_features     ": max_features     ,
                                                "max_depth        ": max_depth        ,
                                                "min_samples_split": min_samples_split,
                                                "min_samples_leaf ": min_samples_leaf 
                                              }),
                               pd.DataFrame(acclist)], axis = 1).reindex())
knnrdf.columns = ['max_depth', 'max_features', 'min_samples_leaf', 'min_samples_split', 'splitter', 'Iteration 0', 'Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6', 'Iteration 7', 'Iteration 8', 'Iteration 9', 'MeanSquaredError', 'RunTime']
display(knnrdf)


acclist = []

splitter           =  'random' 
max_features       =  ['auto', 'auto' , 'auto', 'auto', 'auto', 'auto', 'auto', 14    , 14    , 14    , 14  , 14   , 14   ] 
max_depth          =  [None  , None   , None  , None  , None  , None  , None  , None  , 1000  , 500   , 100 , 1000 , 1000 ] 
min_samples_split  =  [2     , 8      , 12    , 16    , 20    , 50    , 80    , 50    , 50    , 50    , 50  , 50   , 50   ] 
min_samples_leaf   =  [1     , 4      , 6     , 8     , 10    , 25    , 40    , 25    , 25    , 25    , 25  , 25   , 25   ]

for i in range(0,len(max_features)):
    acclist.append(knnr_explor(ScaledData         = myData_scaled_regression,
                               splitter           = splitter, 
                               max_features       = max_features[i], 
                               max_depth          = max_depth[i], 
                               min_samples_split  = min_samples_split[i],
                               min_samples_leaf   = min_samples_leaf[i],
                              )
                  )

knnrdf = pd.DataFrame(pd.concat([pd.DataFrame({
                                                "splitter         ": splitter         ,
                                                "max_features     ": max_features     ,
                                                "max_depth        ": max_depth        ,
                                                "min_samples_split": min_samples_split,
                                                "min_samples_leaf ": min_samples_leaf 
                                              }),
                               pd.DataFrame(acclist)], axis = 1).reindex())
knnrdf.columns = ['max_depth', 'max_features', 'min_samples_leaf', 'min_samples_split', 'splitter', 'Iteration 0', 'Iteration 1', 'Iteration 2', 'Iteration 3', 'Iteration 4', 'Iteration 5', 'Iteration 6', 'Iteration 7', 'Iteration 8', 'Iteration 9', 'MeanSquaredError', 'RunTime']
display(knnrdf)

### Analyze the results using a chosen method of evaluation
#### Classification

We have created a function to be re-used for our cross-validation Accuracy Scores. Inputs of PCA components, Model CLF object, original sample data, and a CV containing our test/train splits allow us to easily produce an array of Accuracy Scores, and log_loss values for the different permutations of models tested. A ROC Curve plot is also displayed depicting a stacked view of the ROC_AUC Values for each iteration. Finally, a confusion matrix is displayed for the last test/train iteration for further interpretation on results. 

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.rcParams['figure.figsize'] = (12, 6)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    plt.show()

In [ ]:
def plot_ROC_curve(X, y, mean_tpr, mean_fpr, cv = cv, ):
    
    plt.rcParams['figure.figsize'] = (12, 6)

    lw = 2

    plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k',
             label='Luck')

    mean_tpr /= cv.get_n_splits(X, y)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, color='g', linestyle='--',
             label='Mean ROC (area = %0.2f)' % mean_auc, lw=lw)

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
%%time

def compute_kfold_scores_Classification( clf,  
                                         ScaledData,
                                         PCA      = pca_classification,
                                         Data     = CitiBikeDataSampled_5050,
                                         cv       = cv):
    
    y = Data['usertype'].values # get the labels we want
    y = np.where(y == 'Subscriber', 1, 0)    
    X = ScaledData


    # Run classifier with cross-validation and plot ROC curves

    # setup pipeline to take PCA, then fit a clf model
    clf_pipe = Pipeline(
        [('PCA',PCA),
         ('CLF',clf)]
    )

    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)

    colors = cycle(['cyan', 'indigo', 'seagreen', 'yellow', 'blue', 'darkorange', 'pink', 'darkred', 'dimgray', 'maroon', 'coral'])
    lw = 2
    i = 0
    accuracy = []
    logloss = []
    
    for (train, test), color in zip(cv.split(X, y), colors):
        clf_pipe.fit(X[train],y[train])  # train object
        y_hat = clf_pipe.predict(X[test]) # get test set preditions
        
        a = float(mt.accuracy_score(y[test],y_hat))
        l = float(mt.log_loss(y[test], y_hat))
        
        accuracy.append(round(a,5)) 

        logloss.append(round(l,5)) 


        probas_ = clf_pipe.fit(X[train], y[train]).predict_proba(X[test])
        
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = auc(fpr, tpr)
        
        plt.rcParams['figure.figsize'] = (12, 6)
        
        plt.plot(fpr, tpr, lw=lw, color=color,
                 label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

        i += 1
    
    print("Accuracy Ratings across all iterations: {0}\n\n\
Average Accuracy: {1}\n\n\
Log Loss Values across all iterations: {2}\n\n\
Average Log Loss: {3}\n".format(accuracy, round(sum(accuracy)/len(accuracy),5), logloss,round(sum(logloss)/len(logloss),5)))
    
    plot_ROC_curve(X, y, mean_tpr, mean_fpr)
    
    ytestnames = np.where(y[test] == 1, 'Subscriber', 'Customer')
    yhatnames  = np.where(y_hat == 1, 'Subscriber', 'Customer')
    print("confusion matrix\n{0}\n".format(pd.crosstab(ytestnames, yhatnames, rownames = ['True'], colnames = ['Predicted'], margins = True)))
        
        # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(confusion_matrix(y[test], y_hat), 
                          classes   =["Customer", "Subscriber"], 
                          normalize =True,
                          title     ='Confusion matrix, with normalization')

##### Logistic Regression - Analyze the Results

In [ ]:
%%time

lr_clf = LogisticRegression(penalty='l2', C=.01, class_weight=None, random_state=seed) # get object

compute_kfold_scores_Classification(clf         = lr_clf,
                                    ScaledData  = myData_scaled_classification
                                   )

##### Random Forest  - Analyze the Results

In [ ]:
%%time

rfc_clf = RandomForestClassifier(n_estimators       = 15, 
                                 max_features       = 14, 
                                 max_depth          = 1000.0, 
                                 min_samples_split  = 50, 
                                 min_samples_leaf   = 25, 
                                 n_jobs             = -1, 
                                 random_state       = seed) # get object
    
compute_kfold_scores_Classification(clf         = rfc_clf,
                                    ScaledData  = myData_scaled_classification
                                   )

##### KNN - Analyze the Results

In [ ]:
%%time

knn_clf = KNeighborsClassifier(n_neighbors = 150, algorithm = 'kd_tree', leaf_size = 50, n_jobs=-1) # get object

compute_kfold_scores_Classification(clf         = knn_clf,
                                    ScaledData  = myData_scaled_classification
                                   )

#### Regression

Build a function text...


In [ ]:
#function code for detailed regression metrics

##### KNN Regression - Analyze the Results

##### Decision Tree Regression - Analyze the Results

##### Multi-Layer Perceptron Regression - Analyze the Results

## Discuss the advantages of each model

#### Classification

Who is the Winner????

#### Regression

Who is the Winner????

### Anyone, do research here for python versions of 2ds unit 7.5 material

*XXXXXX If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course. XXXXXX*

## Modeling and Evaluation Part 6 - Which attributes from the analysis are most important?
*XXXXXX Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task. XXXXXX*

With our models compared and contrasted, it is time to discuss attribute importance in helping us make our classification and regression predictions. It is important to remember that our variables used were not the original data set features themselves but rather linear combinations of our data constituting our principal components. We will be reviewing the importance of these principal components in our models in the form of model coefficients.

Another facet of attribute importance we will discuss is that of PCA loadings. As mentioned when performing our in depth review of PCA for both prediction tasks, loadings are a means by which we may understand the underlying factors each principal component represents.

These things being said, this section consists of four subsections:

1. Classification PCA Importance
2. Classification PCA Loadings
3. Regression PCA Importance
4. Regression PCA Loadings

##### Classification PCA Importance

When performing our detailed PCA for our Customer/Subscriber classifications earlier, we identified the first 14 components as being all that were necessary to describe 80% of the variance in the classification data set. When performing logistic regression using these 14 principal components as inputs, we generated coefficient values by which our components are weighted. The larger the coefficient, the greater the component's influence on positive model response. The larger the value in the negative direction, the greater the component's influence on the negative model response.

The coefficient values for each of our principal components are plotted in the horizontal bar chart below. It is immediately apparent that principal components 3 and 10 are weighted heaviest at about 1.75 and 1.65 respectively. Next greatest weights are those of principal components 4 (0.52), 1 (0.49), and 9 (0.42). Interestingly enough, some components are negatively weighted, the greatest of which is principal component 8 at -0.25. While reviewing our logistic regression explanatory variables' (principal components in this case) coefficient values is helpful, it does not help us contextually understand the significance of our original attributes. For this reason, it is important follow up with a review and interpretation of our top weighted principal components' loadings.

In [ ]:
%%time

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
matplotlib.rc('xtick', labelsize=10) 
matplotlib.rc('ytick', labelsize=10)

idx = ['PC'+str(i) for i in range(1,15)]

plt.barh(range(len(lr_clf.coef_[0])), coefs[0], color = 'DarkTurquoise')
plt.show()

##### Classification PCA Loadings

Now at the individual principal component level, each classification principal component has 89 loadings which may be esteemed as weights or coefficients representing each original attribute. These loadings represent to what extent each attribute fabricates a given principal component, and the relationship between these attributes in context of the principal component under review. Another perspective is that each principal component is describing an underlying factor which is comprised of the heaviest loadings.

Rather than discuss all 14 principal components, we will instead focus on the three principal components with the largest positive coefficients (PC3, PC10, and PC4) as described in the PCA importance plot above as well as the component with the largest coefficient value in the negative direction (PC8).

*Principal Component 3 - Early Birds, Saturdays, and General Travel Days/Times *

The 3rd principal component, and component with the largest positive coefficient in the logistic regression model, seems to mostly describe the relationship between day of the week and time of the day and how it relates to rider type. Given PC3's large positive coefficient value in the model, it is interesting to see the negative correlation between Saturday rides (-0.58) and Morning rides (0.58). We could hypothesize that if riders are spending their time riding during the morning hours (5am-10am), they are commuting more for work than they are weekend joy riding. Supporting this arguement are the modest positive loadings for the weekdays and the negative loading for Sunday as well. Interestingly enough however, evening, midday, and afternoon rides are also negatively loaded. While more midday and afternoon riding among Customer vs. Subscribers might be expected, the negative loading for evening riding is a bit of a surprise. It would be good to keep an eye on this attribute's loading in the other principal components as well.

Note that while the general signs are arbitrary from one principal component to the next, the sign relationship between attribute loadings within the same principal component are important.

In [ ]:
matplotlib.rc('xtick', labelsize=8)

weightsplot = pd.Series(pca_class.components_[2], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Tomato')
plt.show()

*Principal Component 10 - Environmental Conditions and the Days of the Week / Time of Day*

We admit we were excited to see the influence of weather on Customer/Subscriber classifications via PC10. Min, max, and average temperatures appear to play a significant role in determining who is a Subscriber and who is only a Customer (temperature loadings approximately -0.50 to -0.51). Not only are the strong negative temperature loadings interesting, but they are negatively correlated with Saturday and Sunday activity (~0.15 and 0.205 respectively). This might suggest that, for example, as the temperature outside increases, subscriber activity decreases on the weekends but increases weekdays (except Fridays). Not only that but night time Subscriber activity (loading of approximately -0.16) increases as well. Remember, the overall signs are arbitrary while signed relationships between attributes are important.

In [ ]:
weightsplot = pd.Series(pca_class.components_[9], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'DeepSkyBlue')
plt.show()

*Principal Component 4 - Leisure vs. Purpose-Centered Weekend Rides*

PC4 is unique in that it requires a respectable amount of domain knowledge to interpret correctly. The most notable loadings are Saturday's 0.6 and Sunday's approximately -0.76 loadings. These are in direct contrast to one another but both occur on the weekend. Noteworthy is also the fact that Morning riding (~0.21) is positively correlated with Saturday riding this time (It was negatively correlated in PC3, but that PC compared weekend vs. weekday riding whereas PC4 focuses primarily on the weekend alone). We might hypothesize that Subscribers might still use the bike share system on Saturdays to run errands, some of which may tend to take place earlier in the day, while Customers tend to joy ride on Sundays, a day when many people tend to relax and take part in recreational activities and other amusements. What isn't surprising is that the day of the week still plays a signficant role in predicting Subscribers vs. Customers.

In [ ]:
weightsplot = pd.Series(pca_class.components_[3], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Orange')
plt.show()

*Principal Component 8 - Beginning vs. End of the Week*

The last principal component for which we will be discussing loadings is PC8. Unlike the previous three PCs reviewed, this component had a negative coefficient in the logistic regression model. The contrast in Tuesday (approximately -0.5) and Thursday (~0.62) loadings is a bit perplexing at first glance. When taking into account the other days of the week and even time of day, however, it's easier to put these relationships into perspective. For example, Thursdays through Sundays all have positive loadings whereas Mondays through Wednesdays all have negative loadings. Accompanying these day of the week correlations are time of day. Given this first half of the work week vs. second half relationship it would appear that individuals riding more in the beginning of the week ride more midday, afternoon, and evening than those riding late in the week and on the weekends. Those individuals might tend to ride more in the morning (Thursdays especially) and at night.

In [ ]:
weightsplot = pd.Series(pca_class.components_[7], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Green')
plt.show()

One important detail that is hard to miss is that start and end stations play a very small role in these top principal components. None of these locations portray any significant loadings within these PCs. Start and end location coordinates played little role as well, though larger than individual station names (See PC3 and PC10). Therefore, we believe it safe to say that in terms of CitiBike user types classification, these attributes are less important.

## DELETE REMAINING PCA LOADING SECTIONS IN CLASSIFICATION

*Principal Component 1 - Peak vs. Non Peak Commute Times*

The first principal component appears to describe the relationships between peak vs. non-peak ride times when people would likely be riding for commutes between work and home vs. leisure rides midday, afternoon, and on the weekends. Specifically, attribute *TimeOfDay_Midday* has a very high loading value around +0.83 while *TimeOfDay_Midday* and *TimeOfDay_Afternoon* have values around -0.35 and -0.35 respectively. While the signs in general are not important from one principal component to the next, the sign relationship between attribute loadings within a component are important. In the case of PC1 it would seem that midday and afternoon ride times are negatively correlated with evening ride times. This is an interesting but expected relationship that would suggest midday, afternoon, and evening ride behaviors (the original TimeOfDay attribute in general really) are important attributes.

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
matplotlib.rc('xtick', labelsize=8) 
matplotlib.rc('ytick', labelsize=10) 

weightsplot = pd.Series(pca_class.components_[0], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'DarkRed')
plt.show()

print(weightsplot.max() - weightsplot.max()*0.1)
print(weightsplot.index[0])

display([i for i in weightsplot.index if i > weightsplot.max() - weightsplot.max()*0.1])
#pca_classification.fit(myData_scaled_classification)
#var = pca_classification.explained_variance_
#var1 = pca_classification.explained_variance_ratio_
#var2 = np.cumsum(np.round(pca_classification.explained_variance_ratio_, decimals=4))
#idx = ['PC'+str(i) for i in range(1,15)]
#
#PCA_df = {'...Eigenvector...' : pd.Series(var, index = idx),
#          '..Variance Proportion..' : pd.Series(var1, index = idx),
#          '.Variance Cumulative Proportion.' : pd.Series(var2, index = idx)}
#
#PCA_df = pd.DataFrame(PCA_df)
#print('Classification Task Principal Component Breakdown')
#display(PCA_df)

*Principal Component 2 - Lunch and Afternoon Rides*

In [ ]:
weightsplot = pd.Series(pca_class.components_[1], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Red')
plt.show()

*Principal Component 5 - TGIF*

In [ ]:
weightsplot = pd.Series(pca_class.components_[4], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Gold')
plt.show()

*Principal Component 6 - Monday Go Getters, Mid-Week Hump, and Morning Riders*

In [ ]:
weightsplot = pd.Series(pca_class.components_[5], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Yellow')
plt.show()

*Principal Component 7 - Mid-Week Madness*

In [ ]:
weightsplot = pd.Series(pca_class.components_[6], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Lime')
plt.show()

*Principal Component 8 - T Days*

*Principal Component 9 - *

In [ ]:
weightsplot = pd.Series(pca_class.components_[8], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'RoyalBlue')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_class.components_[10], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'SlateBlue')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_class.components_[11], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Indigo')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_class.components_[12], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'DarkOrchid')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_class.components_[13], index=class_att.columns)
weightsplot.plot(kind='bar', color = 'Magenta')
plt.show()

##### Regression PCA Loadings

In [ ]:
lr_variable_names = CitiBikeDataSampled_5050.drop(['starttime', 'stoptime', 'start_station_id', 'start_station_name', 'end_station_id', 'end_station_name', 'usertype','tripdurationLog', 'gender', 'gender_0', 'gender_1', 'gender_2', 'birth year', 'Age', 'tripduration', 'DayOfWeek', 'TimeOfDay', 'LinearDistance'], axis=1).columns.values.tolist() 
lr_variable_names = [i for i in lr_variable_names if "end_station_name_" not in i]

weightsplot = pd.Series(pca_reg.components_[0], index=reg_att.columns)
weightsplot.plot(kind='bar', color = 'DarkRed')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[1], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Red')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[2], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Tomato')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[3], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Orange')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[4], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Gold')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[5], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Yellow')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[6], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Lime')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[7], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Green')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[8], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'RoyalBlue')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[9], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'DeepSkyBlue')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[10], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'SlateBlue')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[11], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Indigo')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[12], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'DarkOrchid')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[13], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Magenta')
plt.show()

In [ ]:
weightsplot = pd.Series(pca_reg.components_[14], index=lr_variable_names)
weightsplot.plot(kind='bar', color = 'Gray')
plt.show()

### Deployment

##### How useful is the model for interested parties?
*XXXXXX (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? XXXXXX*

### Exceptional Work

# Old Work for Reuse. Please Delete this later

### LR Feature Weightings Explained

With our model settled upon for our Logistic Regression, we need to explore the weights associated with each feature to get an idea of how each component impacts the ultimate prediction. Looking at our logistic model directly, we found, interestingly enough, that the highest weight was placed on the latitude of the station where renters began their trips. Having a high positive weight means that subscribers were more likely located at lower latitudes, but based on our heat maps, this seemed to contradict our initial data exploration. Moreover, the next highest weight in terms of magnitude was a specific station, and while it appeared in the top ten percent of our most visited stations, it's odd that it would be so important.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
matplotlib.rc('xtick', labelsize=8) 
matplotlib.rc('ytick', labelsize=10) 

lr_weights_raw = lr_clf_RemABYG.coef_.T # take transpose to make a column vector
lr_variable_names = CitiBikeDataSampled_5050.drop(['starttime','stoptime','start_station_id','start_station_name','end_station_id','end_station_name','usertype','gender','DayOfWeek','TimeOfDay','tripduration','Age','gender_0','gender_1','gender_2','birth year'], axis = 1).columns
lr_weights = zip(lr_weights_raw,lr_variable_names)

lr_weights_top_15_sorted = sorted(lr_weights, key=lambda x: abs(x[0]), reverse=True)[:15]

lr_weights_top_15_sorted_coef = []
lr_weights_top_15_sorted_names = []
for coef, name in lr_weights_top_15_sorted:
    lr_weights_top_15_sorted_coef.append(coef[0])
    lr_weights_top_15_sorted_names.append(name)

weightsplot = pd.Series(lr_clf_RemABYG.coef_[0], index=lr_variable_names)
weightsplot.plot(kind='bar')
plt.show()
    


In [ ]:
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12) 

top_15_weights_plot = pd.Series(lr_weights_top_15_sorted_coef, index=lr_weights_top_15_sorted_names)
top_15_weights_plot.plot(kind='bar')
plt.show()


#for coef, name in zip(lr_weights_raw,lr_variable_names):
#    print(name, 'has weight of', coef[0])

After reviewing our dataset once more, we realized that it was likely our weights were compromised due to the fact that we were dealing with a mix of values and quantities that weren't directly comparable. To get a better understanding of how these features were interacting, we decided to attempt to normalize the features, improving our ability to compare these features across their vastly different scales.

This required a refit using the sklearn Standard Scaler training object, but because we're only concerned with the weights and their interactions, we can easily reuse the sets defined in our ShuffleSplit earlier in our regression. However, because we're dealing with scaled and normalized values, we'll use a lower cost value, in this case 0.05 to reflect the expected decrease in variance.

In [ ]:
from sklearn.preprocessing import StandardScaler

matplotlib.rc('xtick', labelsize=8) 
matplotlib.rc('ytick', labelsize=10) 

scl_obj = StandardScaler()
scl_obj.fit(X_Train_RemABYG)

X_train_scaled = scl_obj.transform(X_Train_RemABYG)
X_test_scaled = scl_obj.transform(x_Test_RemABYG)

scalar_lr_clf = LogisticRegression(penalty='l2', C=0.05) # Earlier this cost was 1.0, because we've normalized the dataset, we now can use a lower cost
scalar_lr_clf.fit(X_train_scaled,y_Train_RemABYG)

scalar_y_hat = scalar_lr_clf.predict(X_test_scaled)

scalar_acc = mt.accuracy_score(y_Test_RemABYG, y_hat_RemABYG)
scalar_conf = mt.confusion_matrix(y_Test_RemABYG, y_hat_RemABYG)
print('accuracy:', scalar_acc)
print(scalar_conf)


scalar_weights = zip(scalar_lr_clf.coef_[0],lr_variable_names)

scalar_weights_top_15_sorted = sorted(scalar_weights, key=lambda x: abs(x[0]), reverse=True)[:15]

scalar_weights_top_15_sorted_coef = []
scalar_weights_top_15_sorted_names = []
for coef, name in scalar_weights_top_15_sorted:
    scalar_weights_top_15_sorted_coef.append(coef)
    scalar_weights_top_15_sorted_names.append(name)

scalar_weights = pd.Series(scalar_lr_clf.coef_[0],index=lr_variable_names)
scalar_weights.plot(kind='bar')
plt.show()


The accuracy and prediction matrix closely resembles our earlier regression and SVM models, meaning that the act of normalization did not greatly impact our model, however it did make understanding the weights much easier and, surprisingly, closely resembled some of our previous hypotheses we posited during our inital data gathering.

Perhaps one unique component of these new weights is that the log transform of the trip duration has a negative weight, nearly 1, with the subscriber class. This implies that subscribers spend very little time on their trip, despite the linear distance between stations having a high positive weight. These two features seem to correlate with our earlier observations that subscribers were typically seen commuting rather than touring using the bikes.

Another interesting aspect is the temperature component created after merging Citi Bike rental data with historical weather data. Both the maximum temperature and the minimum temperature have a negative weight with the subscriber class, however the average temperature has a positive weight. If subscribers are commuting, then it might make sense that as the average temperature for the day rises, they're much more likely to make use of their subscription. As it's possible they commute regardless of weather, extreme weathers are not as likely to impact their riding habits and behaviors.

Finally, a few more quick observations based entirely on weight - saturday and sunday rentals both have negative weights in relationship to subscribers, further building upon the idea that subscribers are also generally commuters, and mornings also have a positive weight in regards to subscriber use. This might mean that subscribers are more likely to ride earlier in the morning on weekdays.

In [ ]:
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12) 

scalar_top_15_weights_plot = pd.Series(scalar_weights_top_15_sorted_coef, index=scalar_weights_top_15_sorted_names)
scalar_top_15_weights_plot.plot(kind='bar')
plt.show()

#zip_vars = zip(scalar_lr_clf.coef_.T,lr_variable_names) # combine attributes
#zip_vars = sorted(zip_vars)

#for coef, name in zip_vars:
#    print(name, 'has weight of', coef[0]) # now print them out

### Pros and Cons of LR vs. SVM
Both Logisitic Regression and Support Vector Machines present advantages as well as disadvantages. As might be expected, which approach is best depends on one's data set and problem statement. Our use of both model types with the same data sample set allows us to break down these pros and cons in detail and determine which produces a better fit given our situation. With this in mind, each model type's general advantages and disadvantages are outlined and discussed below.

**General Advantages**

*Logisitic Regression*
- *Simplicity* - The logistic regression model is built in such a way that independent variables are plotted along an x-axis and single-class classification is represented by 0 or 1 on the y-axis. The logit and cost functions are tuned to accurately fit a logistic curve to the data on this x-y plane. By evaluating P(X) via the logit function, we are able to easily determine whether the class is 0 (Customer), in cases when P < 0.5, or 1 (Subscriber), in cases in which P > 0.5. In other words, logistic regression fits all data points as though they were positioned along a continuous function.

- *Small Dimensionality Applications* - Logistic regression works well when there are a smaller number of dimensions and when these attributes do not necessarily determine the response without considering probability of response.

*SVM*
- *Flexible* - Since SVM is capable of fitting a hyperplane among an infinite number of dimensions to create a boundary between classes of various data points, it allows for seperation of data from varying classes even when data sets are very large or contain many attributes.

- *Robust* - In general, SVM is more robust against outliers since it only considers points near the margin as support vectors.

- *Speed (SVM training with Stochastic Gradient Descent)* - Because stochastic gradient descent processes the data in batches and is less precise in it's batch-by-batch approach to deepest descent but still arrives there nonetheless in the end, it is significantly faster than logistic regression and SVM on its own (less resource instensive).

**General Disadvantages**

*Logistic Regression*
- *Speed* - Logistic regression, though faster than straightforward SVM for our data set, is slower in comparison to training a classfier model using stochastic gradient descent.

- *Sensitivity* - In cases when P = ~0.5, logistic regression has difficulty determining classification.

*SVM*
- *Speed (SVM by itself)* - Fitting a SVM model on its own was very slow for our data set. Even after 14 hours of run-time on a quad-core i7 workstation, not even a single model was completely generated.

**The Winner**

Given the general advantages and disadvantages above, we'd now like to discuss both model approaches with respect to our data. These discussions are centered around the data set version in which only Age, Birth Year, and Gender are removed from the originally modeled set. As mentioned above, linear SVM training with stochastic gradient descent for 3x iterations was much faster for our selected data set (~39s) vs. native logistic regression training (~1 min 24s). This time difference is expected to increase exponentially as the number of rides analyzed increases, giving SVM with stochastic gradient descent the upper edge. <span style="color:green">*+1 for SVM with stochastic gradient descent*</span>

On the other hand, however, logistic regression produced accuracy results of {74.52, 74.49, 74.73} % across the three test/train iterations whereas SVM with stochastic gradient descent exhibited accuracy results of {74.121, 74.339, 74.44} %. This slight increase in accuracy among logistic regression models is an interesting observation, especially given logistic regression's added simplicity vs. SVM modelling. A second measure of accuracy employed throughout this analysis has been log-loss. When comparing log-loss among both model types, we expect log-loss values to correspond with model accuracy measures also. As such, logistic regression log-loss values were {8.80, 8.75, 8.73} for each of the three iterations whereas SVM with stochastic gradient values were {8.94, 8.86, 8.83}. As expected, these results correlate with the accuracy scores discussed. <span style="color:blue">*+1 for logistic regression*</span>

The final comparison worth further discussion is that of efficiency. As mentioned in the general advantages and disadvantages above, logistic regression consists of a simpler model than SVM, especially when factoring in stochastic gradient descent to SVM model training. Logistic regression classifies our CitiBike user types by optimizing the log likelihood function; SVM tries to maximize the margin between the closest data points of two classification regions. The fact that stochastic gradient descent is required given our data set dimensions and large number of observations means the model generation process may be deemed less efficient. In the end, the principles of Occam's Razor apply. Therefore, logistic regression without stochastic gradient descent takes the win for our data set.<span style="color:blue">*+1 for logistic regression*</span>

Given its increased accuracy and efficiency/simplicity, we rate logistic regression as being the better model for user type classification using our sample data set of 500,000 rides.